In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
N_up = 1
nb_dir = '/'.join(os.getcwd().split('/')[:-N_up])
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

                                                            
c = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
     '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'] 

# os.environ["CUDA_VISIBLE_DEVICES"] = "6"

## Setup plotting enviroment

In [ ]:
from __future__ import print_function, division
%matplotlib inline
import numpy as np


import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable
from torch.optim import Optimizer


import collections
import sys
import gzip
import os
import math

import time
import torch.utils.data
from torchvision import transforms, datasets
import scipy.ndimage as ndim

try:
    import cPickle as pickle
except:
    import pickle as pickle

In [ ]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split

from src.utils import Datafeed
from src.datasets import make_spirals
from net_wrappers import MF_BNN_cat
from src.probability import variational_categorical, fixed_probs
from train_BNN import train_VI_classification
from stochastic_resnet_models import arq_uncert_conv2d_resnet

from src.plots import evaluate_train_test, errorfill, evaluate_train_test_at_d

%matplotlib inline

In [ ]:
text_width = 5.50107 # in  --> Confirmed with template explanation

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

fs_m1 = 6
fs = 8
fs_p1 = 8

matplotlib.rc('font', serif='Times')
    
matplotlib.rc('font', size=fs)          # controls default text sizes
matplotlib.rc('axes', titlesize=fs)     # fontsize of the axes title
matplotlib.rc('axes', labelsize=fs)    # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=fs_m1)    # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=fs_m1)    # fontsize of the tick labels
matplotlib.rc('legend', fontsize=fs)    # legend fontsize
matplotlib.rc('figure', titlesize=fs_p1)  # fontsize of the figure title


matplotlib.rc('font',**{'family':'serif','serif':['Palatino']})
matplotlib.rcParams['text.latex.preamble'] = [
       r'\usepackage{siunitx}',   # i need upright \micro symbols, but you need...
       r'\sisetup{detect-all}',   # ...this to force siunitx to actually use your fonts
       r'\usepackage{helvet}',    # set the normal font here
       r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
]  
matplotlib.rc('text', usetex=True)


matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

base_c10 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
            '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

base_c11k = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
            '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#000000']

i = range(len(base_c10))
a = [5]*len(base_c10)

plt.figure(dpi=80)
plt.bar(i,a, color=base_c10)
plt.title('Test')
plt.xlabel('test')
plt.show()

In [ ]:
from src.plots import errorfill

plot_savedir = '../saves/paper_plots/'
datadir = plot_savedir + 'data/'

In [ ]:
save_dir = '../saves/logs/'

In [ ]:
def get_run_mean_std(struct, last_max=False, last_min=False, last_argmax=False, last_argmin=False):
    
    means = []
    stds = []
    for exp in struct:
        
        if last_max:
            exp = exp.max(axis=1)
        elif last_min:
            exp = exp.min(axis=1)
        elif last_argmax:
            exp = exp.argmax(axis=1)
        elif last_argmin:
            exp = exp.argmin(axis=1)
        
        means.append(np.nanmean(exp, axis=0))
        stds.append(np.nanstd(exp, axis=0))
        
    return np.array(means), np.array(stds)

### Generate plot data: scan over depth - MNIST

In [ ]:
experiment_dir = save_dir + 'CNN_BNN_MNIST_cat/' + 'max_depth_scan/'

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.MNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)


In [ ]:
max_depths_MNIST = []
per_max_depth_d_exp_MNIST = []
per_max_depth_d_95th_MNIST = []
per_max_depth_q_MNIST = []
per_max_depth_KL_MNIST = []

# per_max_depth_train_loglike_MNIST = []
# per_max_depth_train_err_MNIST = []
per_max_depth_test_loglike_MNIST = []
per_max_depth_test_err_MNIST = []

# per_max_depth_train_loglike_d_exp_MNIST = []
# per_max_depth_train_err_d_exp_MNIST = []
per_max_depth_test_loglike_d_exp_MNIST = []
per_max_depth_test_err_d_exp_MNIST = []

# per_max_depth_train_loglike_d_95th_MNIST = []
# per_max_depth_train_err_d_95th_MNIST = []
per_max_depth_test_loglike_d_95th_MNIST = []
per_max_depth_test_err_d_95th_MNIST = []

#iterate over experiment settings
exp_folders = os.listdir(experiment_dir)
for exp_folder in exp_folders: # loop through all the files and folders
    full_exp_folder = os.path.join(experiment_dir, exp_folder)
    if os.path.isdir(full_exp_folder): # check whether the current object is a folder or not
        
        if exp_folder[0] == '.':
            continue
        
        print(int(exp_folder))
    
        prior_probs = 0.85 ** (1 + np.arange(int(exp_folder) + 1))
        prior_probs = prior_probs / prior_probs.sum()
        prob_model = variational_categorical(int(exp_folder), prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                     inner_width, int(exp_folder), prob_model)  
        N_train = len(trainset)
        lr = 1e-1
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

        max_depths_MNIST.append(int(exp_folder))
        
        per_max_depth_q_MNIST.append([])
        per_max_depth_KL_MNIST.append([])
        per_max_depth_d_exp_MNIST.append([])
        per_max_depth_d_95th_MNIST.append([])
        # per_max_depth_train_loglike_MNIST.append([])
        # per_max_depth_train_err_MNIST.append([])
        per_max_depth_test_loglike_MNIST.append([])
        per_max_depth_test_err_MNIST.append([])
        
        # per_max_depth_train_loglike_d_exp_MNIST.append([])
        # per_max_depth_train_err_d_exp_MNIST.append([])
        per_max_depth_test_loglike_d_exp_MNIST.append([])
        per_max_depth_test_err_d_exp_MNIST.append([])

        # per_max_depth_train_loglike_d_95th_MNIST.append([])
        # per_max_depth_train_err_d_95th_MNIST.append([])
        per_max_depth_test_loglike_d_95th_MNIST.append([])
        per_max_depth_test_err_d_95th_MNIST.append([])
        
        N_run_folders = os.listdir(full_exp_folder)
        for N_run_folder in N_run_folders: # loop through all the files and folders
            model_folder = os.path.join(full_exp_folder, N_run_folder, 'models')
            if os.path.isdir(model_folder): # check whether the current object is a folder or not
                
                try:
                    net.load(model_folder + '/theta_best.dat')
                except:
                    print('could not load: ' + model_folder)
                    continue
                
                per_max_depth_q_MNIST[-1].append(net.model.prob_model.get_q_probs().data.cpu().numpy())
                per_max_depth_KL_MNIST[-1].append(net.model.get_KL().item())
                
                loglike_tests, err_tests, loglike_tests_exp, err_tests_exp, loglike_tests_95th, err_tests_95th = [], [], [], [], [], []
        
                probs = net.model.prob_model.get_q_probs().data.cpu().numpy()
                cuttoff = np.max(probs)*0.95
                depth_exp = np.sum(probs * np.arange(net.model.n_layers + 1))
                depth_95th = np.argmax(probs > cuttoff)
                per_max_depth_d_exp_MNIST[-1].append(depth_exp)
                per_max_depth_d_95th_MNIST[-1].append(depth_95th)

                for x, y in valloader:
                    if cuda:
                        y = y.cuda()

                    probs = net.sample_predict(x).sum(dim=0)
                    loglike = -F.nll_loss(probs, y, reduction='sum').item()
                    pred = probs.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err = pred.ne(y).sum().item()

                    loglike_tests.append(loglike)
                    err_tests.append(err)

                    probs_exp = net.partial_predict(x, int(round(depth_exp))).sum(dim=0)
                    loglike_exp = -F.nll_loss(probs_exp, y, reduction='sum').item()
                    pred_exp = probs_exp.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err_exp = pred_exp.ne(y).sum().item()

                    loglike_tests_exp.append(loglike_exp)
                    err_tests_exp.append(err_exp)

                    probs_95th = net.partial_predict(x, depth_95th).sum(dim=0)
                    loglike_95th = -F.nll_loss(probs_95th, y, reduction='sum').item()
                    pred_95th = probs_95th.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err_95th = pred_95th.ne(y).sum().item()

                    loglike_tests_95th.append(loglike_95th)
                    err_tests_95th.append(err_95th)

                N_test = len(valset)
                loglike_test = np.sum(loglike_tests) / N_test
                err_test= np.sum(err_tests) / N_test
                loglike_test_exp= np.sum(loglike_tests_exp) / N_test
                err_test_exp = np.sum(err_tests_exp) / N_test
                loglike_test_95th = np.sum(loglike_tests_95th) / N_test
                err_test_95th = np.sum(err_tests_95th) / N_test
                    
                # per_max_depth_train_loglike_MNIST[-1].append(loglike_train)
                # per_max_depth_train_err_MNIST[-1].append(err_train)
                per_max_depth_test_loglike_MNIST[-1].append(loglike_test)
                per_max_depth_test_err_MNIST[-1].append(err_test)
                # per_max_depth_train_loglike_d_exp_MNIST[-1].append(loglike_train_exp)
                # per_max_depth_train_err_d_exp_MNIST[-1].append(err_train_exp)
                per_max_depth_test_loglike_d_exp_MNIST[-1].append(loglike_test_exp)
                per_max_depth_test_err_d_exp_MNIST[-1].append(err_test_exp)
                # per_max_depth_train_loglike_d_95th_MNIST[-1].append(loglike_train_95th)
                # per_max_depth_train_err_d_95th_MNIST[-1].append(err_train_95th)
                per_max_depth_test_loglike_d_95th_MNIST[-1].append(loglike_test_95th)
                per_max_depth_test_err_d_95th_MNIST[-1].append(err_test_95th)
                
        per_max_depth_q_MNIST[-1] = np.stack(per_max_depth_q_MNIST[-1], axis=0)
        per_max_depth_KL_MNIST[-1] = np.array(per_max_depth_KL_MNIST[-1])
        # per_max_depth_train_loglike_MNIST[-1] = np.array(per_max_depth_train_loglike_MNIST[-1])
        # per_max_depth_train_err_MNIST[-1] = np.array(per_max_depth_train_err_MNIST[-1])
        per_max_depth_test_loglike_MNIST[-1] = np.array(per_max_depth_test_loglike_MNIST[-1])
        per_max_depth_test_err_MNIST[-1] = np.array(per_max_depth_test_err_MNIST[-1])
        
        # per_max_depth_train_loglike_d_MNIST[-1] = np.array(per_max_depth_train_loglike_d_MNIST[-1])
        # per_max_depth_train_err_d_MNIST[-1] = np.array(per_max_depth_train_err_d_MNIST[-1])
        per_max_depth_test_loglike_d_exp_MNIST[-1] = np.array(per_max_depth_test_loglike_d_exp_MNIST[-1])
        per_max_depth_test_err_d_exp_MNIST[-1] = np.array(per_max_depth_test_err_d_exp_MNIST[-1])

        # per_max_depth_train_loglike_d_MNIST[-1] = np.array(per_max_depth_train_loglike_d_MNIST[-1])
        # per_max_depth_train_err_d_MNIST[-1] = np.array(per_max_depth_train_err_d_MNIST[-1])
        per_max_depth_test_loglike_d_95th_MNIST[-1] = np.array(per_max_depth_test_loglike_d_95th_MNIST[-1])
        per_max_depth_test_err_d_95th_MNIST[-1] = np.array(per_max_depth_test_err_d_95th_MNIST[-1])     
                       
sort_idxs = np.argsort(max_depths_MNIST)

max_depths_MNIST = np.array(max_depths_MNIST)[sort_idxs]

per_max_depth_q_MNIST = [per_max_depth_q_MNIST[i] for i in sort_idxs]
per_max_depth_KL_MNIST = [per_max_depth_KL_MNIST[i] for i in sort_idxs]
per_max_depth_d_exp_MNIST = [per_max_depth_d_exp_MNIST[i] for i in sort_idxs]
per_max_depth_d_95th_MNIST = [per_max_depth_d_95th_MNIST[i] for i in sort_idxs]

# per_max_depth_train_loglike_MNIST = [per_max_depth_train_loglike_MNIST[i] for i in sort_idxs]
# per_max_depth_train_err_MNIST = [per_max_depth_train_err_MNIST[i] for i in sort_idxs]
per_max_depth_test_loglike_MNIST = [per_max_depth_test_loglike_MNIST[i] for i in sort_idxs]
per_max_depth_test_err_MNIST = [per_max_depth_test_err_MNIST[i] for i in sort_idxs]

# per_max_depth_train_loglike_d_exp_MNIST = [per_max_depth_train_loglike_d_exp_MNIST[i] for i in sort_idxs]
# per_max_depth_train_err_d_exp_MNIST = [per_max_depth_train_err_d_exp_MNIST[i] for i in sort_idxs]
per_max_depth_test_loglike_d_exp_MNIST = [per_max_depth_test_loglike_d_exp_MNIST[i] for i in sort_idxs]
per_max_depth_test_err_d_exp_MNIST = [per_max_depth_test_err_d_exp_MNIST[i] for i in sort_idxs]

# per_max_depth_train_loglike_d_95th_MNIST = [per_max_depth_train_loglike_d_95th_MNIST[i] for i in sort_idxs]
# per_max_depth_train_err_d_95th_MNIST = [per_max_depth_train_err_d_95th_MNIST[i] for i in sort_idxs]
per_max_depth_test_loglike_d_95th_MNIST = [per_max_depth_test_loglike_d_95th_MNIST[i] for i in sort_idxs]
per_max_depth_test_err_d_95th_MNIST = [per_max_depth_test_err_d_95th_MNIST[i] for i in sort_idxs]    

### Generate plot data: scan over depth - SVHN

In [ ]:
experiment_dir = save_dir + 'CNN_BNN_SVHN_cat/' + 'max_depth_scan/'

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 3
output_dim = 10
outer_width = 64
inner_width = 32

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

trainset = datasets.SVHN(root='../data', split="train", download=True, transform=transform_train)
valset = datasets.SVHN(root='../data', split="test", download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
max_depths_SVHN = []
per_max_depth_d_exp_SVHN = []
per_max_depth_d_95th_SVHN = []
per_max_depth_q_SVHN = []
per_max_depth_KL_SVHN = []

# per_max_depth_train_loglike_SVHN = []
# per_max_depth_train_err_SVHN = []
per_max_depth_test_loglike_SVHN = []
per_max_depth_test_err_SVHN = []

# per_max_depth_train_loglike_d_exp_SVHN = []
# per_max_depth_train_err_d_exp_SVHN = []
per_max_depth_test_loglike_d_exp_SVHN = []
per_max_depth_test_err_d_exp_SVHN = []

# per_max_depth_train_loglike_d_95th_SVHN = []
# per_max_depth_train_err_d_95th_SVHN = []
per_max_depth_test_loglike_d_95th_SVHN = []
per_max_depth_test_err_d_95th_SVHN = []

#iterate over experiment settings
exp_folders = os.listdir(experiment_dir)
for exp_folder in exp_folders: # loop through all the files and folders
    full_exp_folder = os.path.join(experiment_dir, exp_folder)
    if os.path.isdir(full_exp_folder): # check whether the current object is a folder or not
        
        if exp_folder[0] == '.':
            continue
        
        print(int(exp_folder))
    
        prior_probs = 0.85 ** (1 + np.arange(int(exp_folder) + 1))
        prior_probs = prior_probs / prior_probs.sum()
        prob_model = variational_categorical(int(exp_folder), prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                     inner_width, int(exp_folder), prob_model)  
        N_train = len(trainset)
        lr = 1e-1
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

        max_depths_SVHN.append(int(exp_folder))
        
        per_max_depth_q_SVHN.append([])
        per_max_depth_KL_SVHN.append([])
        per_max_depth_d_exp_SVHN.append([])
        per_max_depth_d_95th_SVHN.append([])
        # per_max_depth_train_loglike_SVHN.append([])
        # per_max_depth_train_err_SVHN.append([])
        per_max_depth_test_loglike_SVHN.append([])
        per_max_depth_test_err_SVHN.append([])
        
        # per_max_depth_train_loglike_d_exp_SVHN.append([])
        # per_max_depth_train_err_d_exp_SVHN.append([])
        per_max_depth_test_loglike_d_exp_SVHN.append([])
        per_max_depth_test_err_d_exp_SVHN.append([])

        # per_max_depth_train_loglike_d_95th_SVHN.append([])
        # per_max_depth_train_err_d_95th_SVHN.append([])
        per_max_depth_test_loglike_d_95th_SVHN.append([])
        per_max_depth_test_err_d_95th_SVHN.append([])
        
        N_run_folders = os.listdir(full_exp_folder)
        for N_run_folder in N_run_folders: # loop through all the files and folders
            model_folder = os.path.join(full_exp_folder, N_run_folder, 'models')
            if os.path.isdir(model_folder): # check whether the current object is a folder or not
                
                try:
                    net.load(model_folder + '/theta_best.dat')
                except:
                    print('could not load: ' + model_folder)
                    continue
                
                per_max_depth_q_SVHN[-1].append(net.model.prob_model.get_q_probs().data.cpu().numpy())
                per_max_depth_KL_SVHN[-1].append(net.model.get_KL().item())
                
                loglike_tests, err_tests, loglike_tests_exp, err_tests_exp, loglike_tests_95th, err_tests_95th = [], [], [], [], [], []
        
                probs = net.model.prob_model.get_q_probs().data.cpu().numpy()
                cuttoff = np.max(probs)*0.95
                depth_exp = np.sum(probs * np.arange(net.model.n_layers + 1))
                depth_95th = np.argmax(probs > cuttoff)
                per_max_depth_d_exp_SVHN[-1].append(depth_exp)
                per_max_depth_d_95th_SVHN[-1].append(depth_95th)

                for x, y in valloader:
                    if cuda:
                        y = y.cuda()

                    probs = net.sample_predict(x).sum(dim=0)
                    loglike = -F.nll_loss(probs, y, reduction='sum').item()
                    pred = probs.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err = pred.ne(y).sum().item()

                    loglike_tests.append(loglike)
                    err_tests.append(err)

                    probs_exp = net.partial_predict(x, int(round(depth_exp))).sum(dim=0)
                    loglike_exp = -F.nll_loss(probs_exp, y, reduction='sum').item()
                    pred_exp = probs_exp.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err_exp = pred_exp.ne(y).sum().item()

                    loglike_tests_exp.append(loglike_exp)
                    err_tests_exp.append(err_exp)

                    probs_95th = net.partial_predict(x, depth_95th).sum(dim=0)
                    loglike_95th = -F.nll_loss(probs_95th, y, reduction='sum').item()
                    pred_95th = probs_95th.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err_95th = pred_95th.ne(y).sum().item()

                    loglike_tests_95th.append(loglike_95th)
                    err_tests_95th.append(err_95th)

                N_test = len(valset)
                loglike_test = np.sum(loglike_tests) / N_test
                err_test= np.sum(err_tests) / N_test
                loglike_test_exp= np.sum(loglike_tests_exp) / N_test
                err_test_exp = np.sum(err_tests_exp) / N_test
                loglike_test_95th = np.sum(loglike_tests_95th) / N_test
                err_test_95th = np.sum(err_tests_95th) / N_test
                    
                # per_max_depth_train_loglike_SVHN[-1].append(loglike_train)
                # per_max_depth_train_err_SVHN[-1].append(err_train)
                per_max_depth_test_loglike_SVHN[-1].append(loglike_test)
                per_max_depth_test_err_SVHN[-1].append(err_test)
                # per_max_depth_train_loglike_d_exp_SVHN[-1].append(loglike_train_exp)
                # per_max_depth_train_err_d_exp_SVHN[-1].append(err_train_exp)
                per_max_depth_test_loglike_d_exp_SVHN[-1].append(loglike_test_exp)
                per_max_depth_test_err_d_exp_SVHN[-1].append(err_test_exp)
                # per_max_depth_train_loglike_d_95th_SVHN[-1].append(loglike_train_95th)
                # per_max_depth_train_err_d_95th_SVHN[-1].append(err_train_95th)
                per_max_depth_test_loglike_d_95th_SVHN[-1].append(loglike_test_95th)
                per_max_depth_test_err_d_95th_SVHN[-1].append(err_test_95th)
                
        per_max_depth_q_SVHN[-1] = np.stack(per_max_depth_q_SVHN[-1], axis=0)
        per_max_depth_KL_SVHN[-1] = np.array(per_max_depth_KL_SVHN[-1])
        # per_max_depth_train_loglike_SVHN[-1] = np.array(per_max_depth_train_loglike_SVHN[-1])
        # per_max_depth_train_err_SVHN[-1] = np.array(per_max_depth_train_err_SVHN[-1])
        per_max_depth_test_loglike_SVHN[-1] = np.array(per_max_depth_test_loglike_SVHN[-1])
        per_max_depth_test_err_SVHN[-1] = np.array(per_max_depth_test_err_SVHN[-1])
        
        # per_max_depth_train_loglike_d_SVHN[-1] = np.array(per_max_depth_train_loglike_d_SVHN[-1])
        # per_max_depth_train_err_d_SVHN[-1] = np.array(per_max_depth_train_err_d_SVHN[-1])
        per_max_depth_test_loglike_d_exp_SVHN[-1] = np.array(per_max_depth_test_loglike_d_exp_SVHN[-1])
        per_max_depth_test_err_d_exp_SVHN[-1] = np.array(per_max_depth_test_err_d_exp_SVHN[-1])

        # per_max_depth_train_loglike_d_SVHN[-1] = np.array(per_max_depth_train_loglike_d_SVHN[-1])
        # per_max_depth_train_err_d_SVHN[-1] = np.array(per_max_depth_train_err_d_SVHN[-1])
        per_max_depth_test_loglike_d_95th_SVHN[-1] = np.array(per_max_depth_test_loglike_d_95th_SVHN[-1])
        per_max_depth_test_err_d_95th_SVHN[-1] = np.array(per_max_depth_test_err_d_95th_SVHN[-1])   
                       
sort_idxs_SVHN = np.argsort(max_depths_SVHN)

max_depths_SVHN = np.array(max_depths_SVHN)[sort_idxs_SVHN]

per_max_depth_q_SVHN = [per_max_depth_q_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_KL_SVHN = [per_max_depth_KL_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_d_exp_SVHN = [per_max_depth_d_exp_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_d_95th_SVHN = [per_max_depth_d_95th_SVHN[i] for i in sort_idxs_SVHN]

# per_max_depth_train_loglike_SVHN = [per_max_depth_train_loglike_SVHN[i] for i in sort_idxs_SVHN]
# per_max_depth_train_err_SVHN = [per_max_depth_train_err_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_test_loglike_SVHN = [per_max_depth_test_loglike_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_test_err_SVHN = [per_max_depth_test_err_SVHN[i] for i in sort_idxs_SVHN]


# per_max_depth_train_loglike_d_exp_SVHN = [per_max_depth_train_loglike_d_exp_SVHN[i] for i in sort_idxs]
# per_max_depth_train_err_d_exp_SVHN = [per_max_depth_train_err_d_exp_SVHN[i] for i in sort_idxs]
per_max_depth_test_loglike_d_exp_SVHN = [per_max_depth_test_loglike_d_exp_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_test_err_d_exp_SVHN = [per_max_depth_test_err_d_exp_SVHN[i] for i in sort_idxs_SVHN]

# per_max_depth_train_loglike_d_95th_SVHN = [per_max_depth_train_loglike_d_95th_SVHN[i] for i in sort_idxs_SVHN]
# per_max_depth_train_err_d_95th_SVHN = [per_max_depth_train_err_d_95th_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_test_loglike_d_95th_SVHN = [per_max_depth_test_loglike_d_95th_SVHN[i] for i in sort_idxs_SVHN]
per_max_depth_test_err_d_95th_SVHN = [per_max_depth_test_err_d_95th_SVHN[i] for i in sort_idxs_SVHN]       

### Generate plot data: scan over depth - FashionMNIST

In [ ]:
experiment_dir = save_dir + 'CNN_BNN_FashionMNIST_cat/' + 'max_depth_scan/'

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

trainset = datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
max_depths_FashionMNIST = []
per_max_depth_d_exp_FashionMNIST = []
per_max_depth_d_95th_FashionMNIST = []
per_max_depth_q_FashionMNIST = []
per_max_depth_KL_FashionMNIST = []

# per_max_depth_train_loglike_FashionMNIST = []
# per_max_depth_train_err_FashionMNIST = []
per_max_depth_test_loglike_FashionMNIST = []
per_max_depth_test_err_FashionMNIST = []

# per_max_depth_train_loglike_d_exp_FashionMNIST = []
# per_max_depth_train_err_d_exp_FashionMNIST = []
per_max_depth_test_loglike_d_exp_FashionMNIST = []
per_max_depth_test_err_d_exp_FashionMNIST = []

# per_max_depth_train_loglike_d_95th_FashionMNIST = []
# per_max_depth_train_err_d_95th_FashionMNIST = []
per_max_depth_test_loglike_d_95th_FashionMNIST = []
per_max_depth_test_err_d_95th_FashionMNIST = []

#iterate over experiment settings
exp_folders = os.listdir(experiment_dir)
for exp_folder in exp_folders: # loop through all the files and folders
    full_exp_folder = os.path.join(experiment_dir, exp_folder)
    if os.path.isdir(full_exp_folder): # check whether the current object is a folder or not
        
        if exp_folder[0] == '.':
            continue
        
        print(int(exp_folder))
    
        prior_probs = 0.85 ** (1 + np.arange(int(exp_folder) + 1))
        prior_probs = prior_probs / prior_probs.sum()
        prob_model = variational_categorical(int(exp_folder), prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                     inner_width, int(exp_folder), prob_model)  
        N_train = len(trainset)
        lr = 1e-1
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

        max_depths_FashionMNIST.append(int(exp_folder))
        
        per_max_depth_q_FashionMNIST.append([])
        per_max_depth_KL_FashionMNIST.append([])
        per_max_depth_d_exp_FashionMNIST.append([])
        per_max_depth_d_95th_FashionMNIST.append([])
        # per_max_depth_train_loglike_FashionMNIST.append([])
        # per_max_depth_train_err_FashionMNIST.append([])
        per_max_depth_test_loglike_FashionMNIST.append([])
        per_max_depth_test_err_FashionMNIST.append([])
        
        # per_max_depth_train_loglike_d_exp_FashionMNIST.append([])
        # per_max_depth_train_err_d_exp_FashionMNIST.append([])
        per_max_depth_test_loglike_d_exp_FashionMNIST.append([])
        per_max_depth_test_err_d_exp_FashionMNIST.append([])

        # per_max_depth_train_loglike_d_95th_FashionMNIST.append([])
        # per_max_depth_train_err_d_95th_FashionMNIST.append([])
        per_max_depth_test_loglike_d_95th_FashionMNIST.append([])
        per_max_depth_test_err_d_95th_FashionMNIST.append([])
        
        N_run_folders = os.listdir(full_exp_folder)
        for N_run_folder in N_run_folders: # loop through all the files and folders
            model_folder = os.path.join(full_exp_folder, N_run_folder, 'models')
            if os.path.isdir(model_folder): # check whether the current object is a folder or not
                
                try:
                    net.load(model_folder + '/theta_best.dat')
                except:
                    print('could not load: ' + model_folder)
                    continue
                
                per_max_depth_q_FashionMNIST[-1].append(net.model.prob_model.get_q_probs().data.cpu().numpy())
                per_max_depth_KL_FashionMNIST[-1].append(net.model.get_KL().item())
                
                loglike_tests, err_tests, loglike_tests_exp, err_tests_exp, loglike_tests_95th, err_tests_95th = [], [], [], [], [], []
        
                probs = net.model.prob_model.get_q_probs().data.cpu().numpy()
                cuttoff = np.max(probs)*0.95
                depth_exp = np.sum(probs * np.arange(net.model.n_layers + 1))
                depth_95th = np.argmax(probs > cuttoff)
                per_max_depth_d_exp_FashionMNIST[-1].append(depth_exp)
                per_max_depth_d_95th_FashionMNIST[-1].append(depth_95th)

                for x, y in valloader:
                    if cuda:
                        y = y.cuda()

                    probs = net.sample_predict(x).sum(dim=0)
                    loglike = -F.nll_loss(probs, y, reduction='sum').item()
                    pred = probs.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err = pred.ne(y).sum().item()

                    loglike_tests.append(loglike)
                    err_tests.append(err)

                    probs_exp = net.partial_predict(x, int(round(depth_exp))).sum(dim=0)
                    loglike_exp = -F.nll_loss(probs_exp, y, reduction='sum').item()
                    pred_exp = probs_exp.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err_exp = pred_exp.ne(y).sum().item()

                    loglike_tests_exp.append(loglike_exp)
                    err_tests_exp.append(err_exp)

                    probs_95th = net.partial_predict(x, depth_95th).sum(dim=0)
                    loglike_95th = -F.nll_loss(probs_95th, y, reduction='sum').item()
                    pred_95th = probs_95th.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err_95th = pred_95th.ne(y).sum().item()

                    loglike_tests_95th.append(loglike_95th)
                    err_tests_95th.append(err_95th)

                N_test = len(valset)
                loglike_test = np.sum(loglike_tests) / N_test
                err_test= np.sum(err_tests) / N_test
                loglike_test_exp= np.sum(loglike_tests_exp) / N_test
                err_test_exp = np.sum(err_tests_exp) / N_test
                loglike_test_95th = np.sum(loglike_tests_95th) / N_test
                err_test_95th = np.sum(err_tests_95th) / N_test
                    
                # per_max_depth_train_loglike_FashionMNIST[-1].append(loglike_train)
                # per_max_depth_train_err_FashionMNIST[-1].append(err_train)
                per_max_depth_test_loglike_FashionMNIST[-1].append(loglike_test)
                per_max_depth_test_err_FashionMNIST[-1].append(err_test)
                # per_max_depth_train_loglike_d_exp_FashionMNIST[-1].append(loglike_train_exp)
                # per_max_depth_train_err_d_exp_FashionMNIST[-1].append(err_train_exp)
                per_max_depth_test_loglike_d_exp_FashionMNIST[-1].append(loglike_test_exp)
                per_max_depth_test_err_d_exp_FashionMNIST[-1].append(err_test_exp)
                # per_max_depth_train_loglike_d_95th_FashionMNIST[-1].append(loglike_train_95th)
                # per_max_depth_train_err_d_95th_FashionMNIST[-1].append(err_train_95th)
                per_max_depth_test_loglike_d_95th_FashionMNIST[-1].append(loglike_test_95th)
                per_max_depth_test_err_d_95th_FashionMNIST[-1].append(err_test_95th)    
        
        per_max_depth_q_FashionMNIST[-1] = np.stack(per_max_depth_q_FashionMNIST[-1], axis=0)
        per_max_depth_KL_FashionMNIST[-1] = np.array(per_max_depth_KL_FashionMNIST[-1])
        # per_max_depth_train_loglike_FashionMNIST[-1] = np.array(per_max_depth_train_loglike_FashionMNIST[-1])
        # per_max_depth_train_err_FashionMNIST[-1] = np.array(per_max_depth_train_err_FashionMNIST[-1])
        per_max_depth_test_loglike_FashionMNIST[-1] = np.array(per_max_depth_test_loglike_FashionMNIST[-1])
        per_max_depth_test_err_FashionMNIST[-1] = np.array(per_max_depth_test_err_FashionMNIST[-1])
        
        # per_max_depth_train_loglike_d_FashionMNIST[-1] = np.array(per_max_depth_train_loglike_d_FashionMNIST[-1])
        # per_max_depth_train_err_d_FashionMNIST[-1] = np.array(per_max_depth_train_err_d_FashionMNIST[-1])
        per_max_depth_test_loglike_d_exp_FashionMNIST[-1] = np.array(per_max_depth_test_loglike_d_exp_FashionMNIST[-1])
        per_max_depth_test_err_d_exp_FashionMNIST[-1] = np.array(per_max_depth_test_err_d_exp_FashionMNIST[-1])

        # per_max_depth_train_loglike_d_FashionMNIST[-1] = np.array(per_max_depth_train_loglike_d_FashionMNIST[-1])
        # per_max_depth_train_err_d_FashionMNIST[-1] = np.array(per_max_depth_train_err_d_FashionMNIST[-1])
        per_max_depth_test_loglike_d_95th_FashionMNIST[-1] = np.array(per_max_depth_test_loglike_d_95th_FashionMNIST[-1])
        per_max_depth_test_err_d_95th_FashionMNIST[-1] = np.array(per_max_depth_test_err_d_95th_FashionMNIST[-1])
        
                       
sort_idxs_FashionMNIST = np.argsort(max_depths_FashionMNIST)

max_depths_FashionMNIST = np.array(max_depths_FashionMNIST)[sort_idxs_FashionMNIST]

per_max_depth_q_FashionMNIST = [per_max_depth_q_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_KL_FashionMNIST = [per_max_depth_KL_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_d_exp_FashionMNIST = [per_max_depth_d_exp_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_d_95th_FashionMNIST = [per_max_depth_d_95th_FashionMNIST[i] for i in sort_idxs_FashionMNIST]

# per_max_depth_train_loglike_FashionMNIST = [per_max_depth_train_loglike_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
# per_max_depth_train_err_FashionMNIST = [per_max_depth_train_err_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_test_loglike_FashionMNIST = [per_max_depth_test_loglike_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_test_err_FashionMNIST = [per_max_depth_test_err_FashionMNIST[i] for i in sort_idxs_FashionMNIST]


# per_max_depth_train_loglike_d_exp_FashionMNIST = [per_max_depth_train_loglike_d_exp_FashionMNIST[i] for i in sort_idxs]
# per_max_depth_train_err_d_exp_FashionMNIST = [per_max_depth_train_err_d_exp_FashionMNIST[i] for i in sort_idxs]
per_max_depth_test_loglike_d_exp_FashionMNIST = [per_max_depth_test_loglike_d_exp_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_test_err_d_exp_FashionMNIST = [per_max_depth_test_err_d_exp_FashionMNIST[i] for i in sort_idxs_FashionMNIST]

# per_max_depth_train_loglike_d_95th_FashionMNIST = [per_max_depth_train_loglike_d_95th_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
# per_max_depth_train_err_d_95th_FashionMNIST = [per_max_depth_train_err_d_95th_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_test_loglike_d_95th_FashionMNIST = [per_max_depth_test_loglike_d_95th_FashionMNIST[i] for i in sort_idxs_FashionMNIST]
per_max_depth_test_err_d_95th_FashionMNIST = [per_max_depth_test_err_d_95th_FashionMNIST[i] for i in sort_idxs_FashionMNIST]    

### Generate plot data: scan deterministic depth - MNIST

In [ ]:
experiment_dir = save_dir + 'CNN_BNN_MNIST_deterministic_cat/' + 'deterministic_depth_scan/'

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.MNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
d_depths_MNIST = []

# per_d_depth_train_loglike_MNIST = []
# per_d_depth_train_err_MNIST = []
per_d_depth_test_loglike_MNIST = []
per_d_depth_test_err_MNIST = []

#iterate over experiment settings
exp_folders = os.listdir(experiment_dir)
for exp_folder in exp_folders: # loop through all the files and folders
    full_exp_folder = os.path.join(experiment_dir, exp_folder)
    if os.path.isdir(full_exp_folder): # check whether the current object is a folder or not
        
        if exp_folder[0] == '.':
            continue
        
        print(int(exp_folder))
        
        n_layers = int(exp_folder)
        probs = np.zeros(n_layers + 1)
        probs[-1] = 1
        
        prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                     inner_width, int(exp_folder), prob_model)  
        N_train = len(trainset)
        lr = 1e-1
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 
        
        d_depths_MNIST.append(int(exp_folder))
        
        # per_d_depth_train_loglike_MNIST.append([])
        # per_d_depth_train_err_MNIST.append([])
        per_d_depth_test_loglike_MNIST.append([])
        per_d_depth_test_err_MNIST.append([])
        
        N_run_folders = os.listdir(full_exp_folder)
        for N_run_folder in N_run_folders: # loop through all the files and folders
            model_folder = os.path.join(full_exp_folder, N_run_folder, 'models')
            if os.path.isdir(model_folder): # check whether the current object is a folder or not
                
                try:
                    net.load(model_folder + '/theta_best.dat')
                except:
                    print('could not load: ' + model_folder)
                    continue

                loglike_tests, err_tests = [], []

                for x, y in valloader:
                    if cuda:
                        y = y.cuda()

                    probs = net.sample_predict(x).sum(dim=0)
                    loglike = -F.nll_loss(probs, y, reduction='sum').item()
                    pred = probs.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err = pred.ne(y).sum().item()

                    loglike_tests.append(loglike)
                    err_tests.append(err)

                N_test = len(valset)
                loglike_test = np.sum(loglike_tests) / N_test
                err_test= np.sum(err_tests) / N_test
                

            per_d_depth_test_loglike_MNIST[-1].append(loglike_test)
            per_d_depth_test_err_MNIST[-1].append(err_test)

        per_d_depth_test_loglike_MNIST[-1] = np.array(per_d_depth_test_loglike_MNIST[-1])
        per_d_depth_test_err_MNIST[-1] = np.array(per_d_depth_test_err_MNIST[-1])
        
                
sort_idxs_MNIST2 = np.argsort(d_depths_MNIST)

d_depths_MNIST = np.array(d_depths_MNIST)[sort_idxs_MNIST2]

per_d_depth_test_loglike_MNIST = [per_d_depth_test_loglike_MNIST[i] for i in sort_idxs_MNIST2]
per_d_depth_test_err_MNIST = [per_d_depth_test_err_MNIST[i] for i in sort_idxs_MNIST2]   

### Gen plot data: scan deterministic depth - SVHN

In [ ]:
experiment_dir = save_dir + 'CNN_BNN_SVHN_deterministic_cat/' + 'deterministic_depth_scan/'

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 3
output_dim = 10
outer_width = 64
inner_width = 32

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

trainset = datasets.SVHN(root='../data', split="train", download=True, transform=transform_train)
valset = datasets.SVHN(root='../data', split="test", download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
d_depths_SVHN = []

# per_d_depth_train_loglike_SVHN = []
# per_d_depth_train_err_SVHN = []
per_d_depth_test_loglike_SVHN = []
per_d_depth_test_err_SVHN = []

#iterate over experiment settings
exp_folders = os.listdir(experiment_dir)
for exp_folder in exp_folders: # loop through all the files and folders
    full_exp_folder = os.path.join(experiment_dir, exp_folder)
    if os.path.isdir(full_exp_folder): # check whether the current object is a folder or not
        
        if exp_folder[0] == '.':
            continue
        
        print(int(exp_folder))
        
        n_layers = int(exp_folder)
        probs = np.zeros(n_layers + 1)
        probs[-1] = 1
        
        prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                     inner_width, int(exp_folder), prob_model)  
        N_train = len(trainset)
        lr = 1e-1
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 
        
        d_depths_SVHN.append(int(exp_folder))
        
        # per_d_depth_train_loglike_SVHN.append([])
        # per_d_depth_train_err_SVHN.append([])
        per_d_depth_test_loglike_SVHN.append([])
        per_d_depth_test_err_SVHN.append([])
        
        
        N_run_folders = os.listdir(full_exp_folder)
        for N_run_folder in N_run_folders: # loop through all the files and folders
            model_folder = os.path.join(full_exp_folder, N_run_folder, 'models')
            if os.path.isdir(model_folder): # check whether the current object is a folder or not
                
                try:
                    net.load(model_folder + '/theta_best.dat')
                except:
                    print('could not load: ' + model_folder)
                    continue

                loglike_tests, err_tests = [], []

                for x, y in valloader:
                    if cuda:
                        y = y.cuda()

                    probs = net.sample_predict(x).sum(dim=0)
                    loglike = -F.nll_loss(probs, y, reduction='sum').item()
                    pred = probs.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err = pred.ne(y).sum().item()

                    loglike_tests.append(loglike)
                    err_tests.append(err)

                N_test = len(valset)
                loglike_test = np.sum(loglike_tests) / N_test
                err_test= np.sum(err_tests) / N_test
                

            per_d_depth_test_loglike_SVHN[-1].append(loglike_test)
            per_d_depth_test_err_SVHN[-1].append(err_test)

        per_d_depth_test_loglike_SVHN[-1] = np.array(per_d_depth_test_loglike_SVHN[-1])
        per_d_depth_test_err_SVHN[-1] = np.array(per_d_depth_test_err_SVHN[-1])
        
                
sort_idxs_SVHN2 = np.argsort(d_depths_SVHN)

d_depths_SVHN = np.array(d_depths_SVHN)[sort_idxs_SVHN2]

per_d_depth_test_loglike_SVHN = [per_d_depth_test_loglike_SVHN[i] for i in sort_idxs_SVHN2]
per_d_depth_test_err_SVHN = [per_d_depth_test_err_SVHN[i] for i in sort_idxs_SVHN2]   

### Get plot data: scan deterministic depth - FashionMNIST

In [ ]:
experiment_dir = save_dir + 'CNN_BNN_FashionMNIST_deterministic_cat/' + 'deterministic_depth_scan/'

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

trainset = datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
d_depths_FashionMNIST = []

# per_d_depth_train_loglike_FashionMNIST = []
# per_d_depth_train_err_FashionMNIST = []
per_d_depth_test_loglike_FashionMNIST = []
per_d_depth_test_err_FashionMNIST = []

#iterate over experiment settings
exp_folders = os.listdir(experiment_dir)
for exp_folder in exp_folders: # loop through all the files and folders
    full_exp_folder = os.path.join(experiment_dir, exp_folder)
    if os.path.isdir(full_exp_folder): # check whether the current object is a folder or not
        
        if exp_folder[0] == '.':
            continue
        
        print(int(exp_folder))
        
        n_layers = int(exp_folder)
        probs = np.zeros(n_layers + 1)
        probs[-1] = 1
        
        prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                     inner_width, int(exp_folder), prob_model)  
        N_train = len(trainset)
        lr = 1e-1
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 
        
        
        
        d_depths_FashionMNIST.append(int(exp_folder))
        
        # per_d_depth_train_loglike_FashionMNIST.append([])
        # per_d_depth_train_err_FashionMNIST.append([])
        per_d_depth_test_loglike_FashionMNIST.append([])
        per_d_depth_test_err_FashionMNIST.append([])
        
        
        N_run_folders = os.listdir(full_exp_folder)
        for N_run_folder in N_run_folders: # loop through all the files and folders
            model_folder = os.path.join(full_exp_folder, N_run_folder, 'models')
            if os.path.isdir(model_folder): # check whether the current object is a folder or not
                
                try:
                    net.load(model_folder + '/theta_best.dat')
                except:
                    print('could not load: ' + model_folder)
                    continue

                loglike_tests, err_tests = [], []

                for x, y in valloader:
                    if cuda:
                        y = y.cuda()

                    probs = net.sample_predict(x).sum(dim=0)
                    loglike = -F.nll_loss(probs, y, reduction='sum').item()
                    pred = probs.max(dim=1, keepdim=False)[1]  # get the index of the max probability
                    err = pred.ne(y).sum().item()

                    loglike_tests.append(loglike)
                    err_tests.append(err)

                N_test = len(valset)
                loglike_test = np.sum(loglike_tests) / N_test
                err_test= np.sum(err_tests) / N_test
                

            per_d_depth_test_loglike_FashionMNIST[-1].append(loglike_test)
            per_d_depth_test_err_FashionMNIST[-1].append(err_test)

        per_d_depth_test_loglike_FashionMNIST[-1] = np.array(per_d_depth_test_loglike_FashionMNIST[-1])
        per_d_depth_test_err_FashionMNIST[-1] = np.array(per_d_depth_test_err_FashionMNIST[-1])
        
                
sort_idxs_FashionMNIST2 = np.argsort(d_depths_FashionMNIST)

d_depths_FashionMNIST = np.array(d_depths_FashionMNIST)[sort_idxs_FashionMNIST2]


per_d_depth_test_loglike_FashionMNIST = [per_d_depth_test_loglike_FashionMNIST[i] for i in sort_idxs_FashionMNIST2]
per_d_depth_test_err_FashionMNIST = [per_d_depth_test_err_FashionMNIST[i] for i in sort_idxs_FashionMNIST2]
 

### Save plot data

In [ ]:
# to_save = (
#     per_d_depth_test_loglike_MNIST,
#     per_d_depth_test_err_MNIST,
#     per_max_depth_q_MNIST,
#     per_max_depth_test_loglike_MNIST,
#     per_max_depth_test_err_MNIST,
#     per_max_depth_test_loglike_d_exp_MNIST,
#     per_max_depth_test_loglike_d_95th_MNIST,
#     per_max_depth_test_err_d_exp_MNIST,
#     per_max_depth_test_err_d_95th_MNIST,
#     per_max_depth_d_exp_MNIST,
#     per_max_depth_d_95th_MNIST,
#     per_d_depth_test_loglike_FashionMNIST,
#     per_d_depth_test_err_FashionMNIST,
#     per_max_depth_q_FashionMNIST,
#     per_max_depth_test_loglike_FashionMNIST,
#     per_max_depth_test_err_FashionMNIST,
#     per_max_depth_test_loglike_d_exp_FashionMNIST,
#     per_max_depth_test_loglike_d_95th_FashionMNIST,
#     per_max_depth_test_err_d_exp_FashionMNIST,
#     per_max_depth_test_err_d_95th_FashionMNIST,
#     per_max_depth_d_exp_FashionMNIST,
#     per_max_depth_d_95th_FashionMNIST,
#     per_d_depth_test_loglike_SVHN,
#     per_d_depth_test_err_SVHN,
#     per_max_depth_q_SVHN,
#     per_max_depth_test_loglike_SVHN,
#     per_max_depth_test_err_SVHN,
#     per_max_depth_test_loglike_d_exp_SVHN,
#     per_max_depth_test_loglike_d_95th_SVHN,
#     per_max_depth_test_err_d_exp_SVHN,
#     per_max_depth_test_err_d_95th_SVHN,
#     per_max_depth_d_exp_SVHN,
#     per_max_depth_d_95th_SVHN,
# )


# with open('../saves/paper_plots/image_depth_dist_exps.pkl','wb') as f:
#      pickle.dump(to_save, f)


### Load plot data

In [ ]:
with open('../saves/paper_plots/image_depth_dist_exps_final.pkl','rb') as f:
     (
        per_d_depth_test_loglike_MNIST,
        per_d_depth_test_err_MNIST,
        per_max_depth_q_MNIST,
        per_max_depth_test_loglike_MNIST,
        per_max_depth_test_err_MNIST,
        per_max_depth_test_loglike_d_exp_MNIST,
        per_max_depth_test_loglike_d_95th_MNIST,
        per_max_depth_test_err_d_exp_MNIST,
        per_max_depth_test_err_d_95th_MNIST,
        per_max_depth_d_exp_MNIST,
        per_max_depth_d_95th_MNIST,
        per_d_depth_test_loglike_FashionMNIST,
        per_d_depth_test_err_FashionMNIST,
        per_max_depth_q_FashionMNIST,
        per_max_depth_test_loglike_FashionMNIST,
        per_max_depth_test_err_FashionMNIST,
        per_max_depth_test_loglike_d_exp_FashionMNIST,
        per_max_depth_test_loglike_d_95th_FashionMNIST,
        per_max_depth_test_err_d_exp_FashionMNIST,
        per_max_depth_test_err_d_95th_FashionMNIST,
        per_max_depth_d_exp_FashionMNIST,
        per_max_depth_d_95th_FashionMNIST,
        per_d_depth_test_loglike_SVHN,
        per_d_depth_test_err_SVHN,
        per_max_depth_q_SVHN,
        per_max_depth_test_loglike_SVHN,
        per_max_depth_test_err_SVHN,
        per_max_depth_test_loglike_d_exp_SVHN,
        per_max_depth_test_loglike_d_95th_SVHN,
        per_max_depth_test_err_d_exp_SVHN,
        per_max_depth_test_err_d_95th_SVHN,
        per_max_depth_d_exp_SVHN,
        per_max_depth_d_95th_SVHN,
    ) = pickle.load(f)

### Generate plots

In [ ]:
n_layers = 50
index = 10

## MNIST
# deterministic
perdd_te_ll_means_MNIST, perdd_te_ll_stds_MNIST = get_run_mean_std(per_d_depth_test_loglike_MNIST)
perdd_te_err_means_MNIST, perdd_te_err_stds_MNIST = get_run_mean_std(np.array(per_d_depth_test_err_MNIST)*100)


# Proposed full depth
depth_means_MNIST, depth_stds_MNIST = get_run_mean_std(per_max_depth_q_MNIST, last_argmax=True)
te_ll_means_MNIST, te_ll_stds_MNIST = get_run_mean_std(per_max_depth_test_loglike_MNIST)
te_err_means_MNIST, te_err_stds_MNIST = get_run_mean_std(np.array(per_max_depth_test_err_MNIST)*100)


# Prune depth
te_ll_means_d_exp_MNIST, te_ll_stds_d_exp_MNIST = get_run_mean_std(per_max_depth_test_loglike_d_exp_MNIST)
te_ll_means_d_MNIST, te_ll_stds_d_MNIST = get_run_mean_std(per_max_depth_test_loglike_d_95th_MNIST)
te_err_means_d_exp_MNIST, te_err_stds_d_exp_MNIST = get_run_mean_std(np.array(per_max_depth_test_err_d_exp_MNIST)*100)
te_err_means_d_MNIST, te_err_stds_d_MNIST = get_run_mean_std(np.array(per_max_depth_test_err_d_95th_MNIST)*100)

depth_means_exp_MNIST, depth_stds_exp_MNIST = get_run_mean_std(per_max_depth_d_exp_MNIST)
depth_means_MNIST, depth_stds_MNIST = get_run_mean_std(per_max_depth_d_95th_MNIST)


# get results at depth n
depth_n_probs_MNIST = per_max_depth_q_MNIST[index].mean(axis=0)
depth_n_probs_std_MNIST = per_max_depth_q_MNIST[index].std(axis=0)

learnt_depth_ll_mean_MNIST = te_ll_means_d_MNIST[index]
learnt_depth_ll_std_MNIST = te_ll_stds_d_MNIST[index]
learnt_depth_err_mean_MNIST = te_err_means_d_MNIST[index]
learnt_depth_err_std_MNIST = te_err_stds_d_MNIST[index]
learnt_depth_mean_MNIST = depth_means_MNIST[index]
learnt_depth_std_MNIST = depth_stds_MNIST[index]

## FashionMNIST
# deterministic
perdd_te_ll_means_FashionMNIST, perdd_te_ll_stds_FashionMNIST = get_run_mean_std(per_d_depth_test_loglike_FashionMNIST)
perdd_te_err_means_FashionMNIST, perdd_te_err_stds_FashionMNIST = get_run_mean_std(np.array(per_d_depth_test_err_FashionMNIST)*100)

# Proposed full depth
depth_means_FashionMNIST, depth_stds_FashionMNIST = get_run_mean_std(per_max_depth_q_FashionMNIST, last_argmax=True)
te_ll_means_FashionMNIST, te_ll_stds_FashionMNIST = get_run_mean_std(per_max_depth_test_loglike_FashionMNIST)
te_err_means_FashionMNIST, te_err_stds_FashionMNIST = get_run_mean_std(np.array(per_max_depth_test_err_FashionMNIST)*100)

# Prune depth
te_ll_means_d_exp_FashionMNIST, te_ll_stds_d_exp_FashionMNIST = get_run_mean_std(per_max_depth_test_loglike_d_exp_FashionMNIST)
te_ll_means_d_FashionMNIST, te_ll_stds_d_FashionMNIST = get_run_mean_std(per_max_depth_test_loglike_d_95th_FashionMNIST)
te_err_means_d_exp_FashionMNIST, te_err_stds_d_exp_FashionMNIST = get_run_mean_std(np.array(per_max_depth_test_err_d_exp_FashionMNIST)*100)
te_err_means_d_FashionMNIST, te_err_stds_d_FashionMNIST = get_run_mean_std(np.array(per_max_depth_test_err_d_95th_FashionMNIST)*100)

depth_means_exp_FashionMNIST, depth_stds_exp_FashionMNIST = get_run_mean_std(per_max_depth_d_exp_FashionMNIST)
depth_means_FashionMNIST, depth_stds_FashionMNIST = get_run_mean_std(per_max_depth_d_95th_FashionMNIST)

# get results at depth n
depth_n_probs_FashionMNIST = per_max_depth_q_FashionMNIST[index].mean(axis=0)
depth_n_probs_std_FashionMNIST = per_max_depth_q_FashionMNIST[index].std(axis=0)

learnt_depth_ll_mean_FashionMNIST = te_ll_means_d_FashionMNIST[index]
learnt_depth_ll_std_FashionMNIST = te_ll_stds_d_FashionMNIST[index]
learnt_depth_err_mean_FashionMNIST = te_err_means_d_FashionMNIST[index]
learnt_depth_err_std_FashionMNIST = te_err_stds_d_FashionMNIST[index]
learnt_depth_mean_FashionMNIST = depth_means_FashionMNIST[index]
learnt_depth_std_FashionMNIST = depth_stds_FashionMNIST[index]

## SVHN
# deterministic
perdd_te_ll_means_SVHN, perdd_te_ll_stds_SVHN = get_run_mean_std(per_d_depth_test_loglike_SVHN)
perdd_te_err_means_SVHN, perdd_te_err_stds_SVHN = get_run_mean_std(np.array(per_d_depth_test_err_SVHN)*100)

# Proposed full depth
depth_means_SVHN, depth_stds_SVHN = get_run_mean_std(per_max_depth_q_SVHN, last_argmax=True)
te_ll_means_SVHN, te_ll_stds_SVHN = get_run_mean_std(per_max_depth_test_loglike_SVHN)
te_err_means_SVHN, te_err_stds_SVHN = get_run_mean_std(np.array(per_max_depth_test_err_SVHN)*100)

# Prune depth
te_ll_means_d_exp_SVHN, te_ll_stds_d_exp_SVHN = get_run_mean_std(per_max_depth_test_loglike_d_exp_SVHN)
te_ll_means_d_SVHN, te_ll_stds_d_SVHN = get_run_mean_std(per_max_depth_test_loglike_d_95th_SVHN)
te_err_means_d_exp_SVHN, te_err_stds_d_exp_SVHN = get_run_mean_std(np.array(per_max_depth_test_err_d_exp_SVHN)*100)
te_err_means_d_SVHN, te_err_stds_d_SVHN = get_run_mean_std(np.array(per_max_depth_test_err_d_95th_SVHN)*100)

depth_means_exp_SVHN, depth_stds_exp_SVHN = get_run_mean_std(per_max_depth_d_exp_SVHN)
depth_means_SVHN, depth_stds_SVHN = get_run_mean_std(per_max_depth_d_95th_SVHN)

# get results at depth n
depth_n_probs_SVHN = per_max_depth_q_SVHN[index].mean(axis=0)
depth_n_probs_std_SVHN = per_max_depth_q_SVHN[index].std(axis=0)

learnt_depth_ll_mean_SVHN = te_ll_means_d_SVHN[index]
learnt_depth_ll_std_SVHN = te_ll_stds_d_SVHN[index]
learnt_depth_err_mean_SVHN = te_err_means_d_SVHN[index]
learnt_depth_err_std_SVHN = te_err_stds_d_SVHN[index]
learnt_depth_mean_SVHN = depth_means_SVHN[index]
learnt_depth_std_SVHN = depth_stds_SVHN[index]

### Main text detailed plot

In [ ]:
markersize = 0.05
lw = 1
s=8
dpi=800 

fig, ax = plt.subplots(nrows=1, ncols=3, dpi=dpi, figsize=(1.1*text_width, 2.18), sharey=True)

ax02 = ax[0].twinx()
ax12 = ax[1].twinx()
ax22 = ax[2].twinx()

ax02.get_shared_y_axes().join(ax02, ax12, ax22)
ax02.get_yaxis().set_ticklabels([])
ax12.get_yaxis().set_ticklabels([])
ax02.set_ylim([0.4, 1])

ax[1].set_xlabel('$d$  (No. active blocks)')
ax[0].set_ylabel(r'$q_{\boldsymbol{\alpha}}(d)$')

ax22.set_ylabel('test log-like')

ax[0].set_title("MNIST")
ax[1].set_title("Fashion-MNIST")
ax[2].set_title("SVHN")

ax[0].set_ylim([0, 0.055])

ax[0].set_xlim([0, 50])
ax[1].set_xlim([0, 50])
ax[2].set_xlim([0, 50])

depths = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]

######### PLOT 1 - MNIST ########################

ax[0].axvspan(learnt_depth_mean_MNIST - learnt_depth_std_MNIST, learnt_depth_mean_MNIST + learnt_depth_std_MNIST, alpha=0.5, facecolor='black')
learnt_depth_line =  ax[0].axvline(learnt_depth_mean_MNIST, color='black', lw=0.7)

bars = ax[0].bar(range(n_layers + 1), depth_n_probs_MNIST, width=(1/n_layers + 1), yerr=depth_n_probs_std_MNIST,
        alpha=0.8, label='$\alpha_{i}$', color=c[0], edgecolor='k', error_kw=dict(lw=0.7))

our_line, = errorfill(range(n_layers + 1), np.array([learnt_depth_ll_mean_MNIST]*(n_layers + 1)),
          np.array([learnt_depth_ll_std_MNIST]*(n_layers + 1)),
          color=c[0], alpha_fill=0.3, ax=ax02, lw=lw, linestyle='-.')

det_depth_line, = errorfill(depths[:index+1], perdd_te_ll_means_MNIST[:index+1], te_ll_stds_MNIST[:index+1],
          color=c[3], alpha_fill=0.3, ax=ax02, lw=lw, linestyle='--')

ax02.scatter(depths[:index+1], perdd_te_ll_means_MNIST[:index+1], c=c[3], s=s)

ax[0].yaxis.grid(alpha=0.3)
ax[0].xaxis.grid(alpha=0.3) 

p_lgd = [ our_line, det_depth_line, bars, learnt_depth_line]
lgd = ax[0].legend(p_lgd,
['LDN', 'DDN', '$\\alpha_{i}$', '$d_{opt}$'], #
                prop={'size': fs-2, 'weight': 'normal'}, frameon=True, markerscale=markersize, loc="lower right")

######### PLOT 2 - FashionMNIST ########################

ax[1].axvspan(learnt_depth_mean_FashionMNIST - learnt_depth_std_FashionMNIST, learnt_depth_mean_FashionMNIST + learnt_depth_std_FashionMNIST, alpha=0.5, facecolor='black')
ax[1].axvline(learnt_depth_mean_FashionMNIST, color='black', lw=0.7)

bars = ax[1].bar(range(n_layers + 1), depth_n_probs_FashionMNIST, width=(1/n_layers + 1), yerr=depth_n_probs_std_FashionMNIST,
        alpha=0.8, label='$\alpha_{i}$', color=c[0], edgecolor='k', error_kw=dict(lw=0.7))

our_line, = errorfill(range(n_layers + 1), np.array([learnt_depth_ll_mean_FashionMNIST]*(n_layers + 1)),
          np.array([learnt_depth_ll_std_FashionMNIST]*(n_layers + 1)),
          color=c[0], alpha_fill=0.3, ax=ax12, lw=lw, linestyle='-.')

det_depth_line, = errorfill(depths[:index+1], perdd_te_ll_means_FashionMNIST[:index+1], te_ll_stds_FashionMNIST[:index+1],
          color=c[3], alpha_fill=0.3, ax=ax12, lw=lw, linestyle='--')

ax12.scatter(depths[:index+1], perdd_te_ll_means_FashionMNIST[:index+1], c=c[3], s=s)

ax[1].yaxis.grid(alpha=0.3)
ax[1].xaxis.grid(alpha=0.3) 

######### PLOT 3 - SVHN ########################

ax[2].axvspan(learnt_depth_mean_SVHN - learnt_depth_std_SVHN, learnt_depth_mean_SVHN + learnt_depth_std_SVHN, alpha=0.5, facecolor='black')
ax[2].axvline(learnt_depth_mean_SVHN, color='black', lw=0.7)

bars = ax[2].bar(range(n_layers + 1), depth_n_probs_SVHN, width=(1/n_layers + 1), yerr=depth_n_probs_std_SVHN,
        alpha=0.8, label='$\alpha_{i}$', color=c[0], edgecolor='k', error_kw=dict(lw=0.7))

our_line, = errorfill(range(n_layers + 1), np.array([learnt_depth_ll_mean_SVHN]*(n_layers + 1)),
          np.array([learnt_depth_ll_std_SVHN]*(n_layers + 1)),
          color=c[0], alpha_fill=0.3, ax=ax22, lw=lw, linestyle='-.')

det_depth_line, = errorfill(depths[:index+1], perdd_te_ll_means_SVHN[:index+1], perdd_te_ll_stds_SVHN[:index+1],
          color=c[3], alpha_fill=0.3, ax=ax22, lw=lw, linestyle='--')

ax22.scatter(depths[:index+1], perdd_te_ll_means_SVHN[:index+1], c=c[3], s=s)

ax[2].yaxis.grid(alpha=0.3)
ax[2].xaxis.grid(alpha=0.3) 


plt.tight_layout()
plt.savefig(plot_savedir + 'image_depth_dist_exps.pdf', format='pdf', bbox_inches='tight')


### Appendix detailed results plot

In [ ]:
markersize = 0.05
lw = 1
s=8
dpi=800

fig, ax = plt.subplots(nrows=5, ncols=3, dpi=dpi, figsize=(1.1*text_width*0.9, 10.*0.9), sharey=False)

ax[4, 1].set_xlabel('D  (Max depth)')

ax[0, 0].set_ylabel('d  (No. active blocks)')
ax[1, 0].set_ylabel('test log-like')
ax[2, 0].set_ylabel('test log-like')
ax[3, 0].set_ylabel('test error')
ax[4, 0].set_ylabel('test error')

ax[0, 0].set_title("MNIST")
ax[0, 1].set_title("Fashion-MNIST")
ax[0, 2].set_title("SVHN")

ax[0, 0].get_shared_y_axes().join(ax[0, 0], ax[0, 1], ax[0, 2])

ax[1, 0].get_shared_y_axes().join(ax[1, 0], ax[1, 1], ax[1, 2])

ax[2, 0].get_shared_y_axes().join(ax[2, 0], ax[2, 1], ax[2, 2])

ax[0, 0].set_ylim([0, 100])
ax[1, 0].set_ylim([0.7, 1])
ax[2, 0].set_ylim([0.7, 1])
ax[3, 0].set_ylim([0, 5])
ax[3, 1].set_ylim([7.5, 20])
ax[3, 2].set_ylim([4, 12])
ax[4, 0].set_ylim([0, 5])
ax[4, 1].set_ylim([7.5, 20])
ax[4, 2].set_ylim([4, 12])

ax[0, 0].set_xlim([0, 100])
ax[0, 1].set_xlim([0, 100])
ax[0, 2].set_xlim([0, 100])
ax[1, 0].set_xlim([0, 100])
ax[1, 1].set_xlim([0, 100])
ax[1, 2].set_xlim([0, 100])
ax[2, 0].set_xlim([0, 100])
ax[2, 1].set_xlim([0, 100])
ax[2, 2].set_xlim([0, 100])
ax[3, 0].set_xlim([0, 100])
ax[3, 1].set_xlim([0, 100])
ax[3, 2].set_xlim([0, 100])
ax[4, 0].set_xlim([0, 100])
ax[4, 1].set_xlim([0, 100])
ax[4, 2].set_xlim([0, 100])

ax[0, 0].yaxis.grid(alpha=0.3)
ax[0, 0].xaxis.grid(alpha=0.3) 
ax[0, 1].yaxis.grid(alpha=0.3)
ax[0, 1].xaxis.grid(alpha=0.3) 
ax[0, 2].yaxis.grid(alpha=0.3)
ax[0, 2].xaxis.grid(alpha=0.3) 
ax[1, 0].yaxis.grid(alpha=0.3)
ax[1, 0].xaxis.grid(alpha=0.3) 
ax[1, 1].yaxis.grid(alpha=0.3)
ax[1, 1].xaxis.grid(alpha=0.3) 
ax[1, 2].yaxis.grid(alpha=0.3)
ax[1, 2].xaxis.grid(alpha=0.3)
ax[2, 0].yaxis.grid(alpha=0.3)
ax[2, 0].xaxis.grid(alpha=0.3) 
ax[2, 1].yaxis.grid(alpha=0.3)
ax[2, 1].xaxis.grid(alpha=0.3) 
ax[2, 2].yaxis.grid(alpha=0.3)
ax[2, 2].xaxis.grid(alpha=0.3) 
ax[3, 0].yaxis.grid(alpha=0.3)
ax[3, 0].xaxis.grid(alpha=0.3) 
ax[3, 1].yaxis.grid(alpha=0.3)
ax[3, 1].xaxis.grid(alpha=0.3) 
ax[3, 2].yaxis.grid(alpha=0.3)
ax[3, 2].xaxis.grid(alpha=0.3) 
ax[4, 0].yaxis.grid(alpha=0.3)
ax[4, 0].xaxis.grid(alpha=0.3) 
ax[4, 1].yaxis.grid(alpha=0.3)
ax[4, 1].xaxis.grid(alpha=0.3) 
ax[4, 2].yaxis.grid(alpha=0.3)
ax[4, 2].xaxis.grid(alpha=0.3)  

depths = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]

######################################
plt.autoscale(enable=True, axis='x', tight=True)

### depths 
## deterministic
det_depths_line, = ax[0, 0].plot(range(np.min(depths), depths[-1]), range(np.min(depths), depths[-1]), '-', c=c[3], lw=lw)
ax[0, 1].plot(range(np.min(depths), depths[-1]), range(np.min(depths), depths[-1]), '-', c=c[3], lw=lw)
ax[0, 2].plot(range(np.min(depths), depths[-1]), range(np.min(depths), depths[-1]), '-', c=c[3], lw=lw)

## learnt 95th
depths_95th_line, = errorfill(depths, depth_means_MNIST, depth_stds_MNIST, color=c[2], alpha_fill=0.3,
                          ax=ax[0, 0], lw=lw, linestyle='-')
ax[0, 0].scatter(depths, depth_means_MNIST, c=c[2], s=s)

errorfill(depths, depth_means_FashionMNIST, depth_stds_FashionMNIST, color=c[2], alpha_fill=0.3,
                          ax=ax[0, 1], lw=lw, linestyle='-')
ax[0, 1].scatter(depths, depth_means_FashionMNIST, c=c[2], s=s)

errorfill(depths, depth_means_SVHN, depth_stds_SVHN, color=c[2], alpha_fill=0.3,
                          ax=ax[0, 2], lw=lw, linestyle='-')
ax[0, 2].scatter(depths, depth_means_SVHN, c=c[2], s=s)

## learnt exp
depths_exp_line, = errorfill(depths, depth_means_exp_MNIST, depth_stds_exp_MNIST, color=c[4], alpha_fill=0.3,
                          ax=ax[0, 0], lw=lw, linestyle='-')
ax[0, 0].scatter(depths, depth_means_exp_MNIST, c=c[4], s=s)

errorfill(depths, depth_means_exp_FashionMNIST, depth_stds_exp_FashionMNIST, color=c[4], alpha_fill=0.3,
                          ax=ax[0, 1], lw=lw, linestyle='-')
ax[0, 1].scatter(depths, depth_means_exp_FashionMNIST, c=c[4], s=s)

errorfill(depths, depth_means_exp_SVHN, depth_stds_exp_SVHN, color=c[4], alpha_fill=0.3,
                          ax=ax[0, 2], lw=lw, linestyle='-')
ax[0, 2].scatter(depths, depth_means_exp_SVHN, c=c[4], s=s)

a_lgd = [det_depths_line, depths_exp_line, depths_95th_line]
lgd = ax[0, 2].legend(a_lgd,
['D', 'LDN-$\mathbb{E}$', 'LDN-95'],
                prop={'size': fs-1, 'weight': 'normal'}, frameon=True, markerscale=markersize, loc='upper left')


### lls
## determinisitc
mnist_det_ll_errrfill, = errorfill(depths, perdd_te_ll_means_MNIST, perdd_te_ll_stds_MNIST, color=c[3], alpha_fill=0.3, ax=ax[1, 0], lw=lw,
                           linestyle='-')
ax[1, 0].scatter(depths, perdd_te_ll_means_MNIST, c=c[3], s=s)

fmnist_det_ll_errrfill, = errorfill(depths, perdd_te_ll_means_FashionMNIST, perdd_te_ll_stds_FashionMNIST, color=c[3], alpha_fill=0.3, ax=ax[1, 1], lw=lw,
                           linestyle='-')
ax[1, 1].scatter(depths, perdd_te_ll_means_FashionMNIST, c=c[3], s=s)

svhn_det_ll_errrfill, = errorfill(depths, perdd_te_ll_means_SVHN, perdd_te_ll_stds_SVHN, color=c[3], alpha_fill=0.3, ax=ax[1, 2], lw=lw,
                           linestyle='-')
ax[1, 2].scatter(depths, perdd_te_ll_means_SVHN, c=c[3], s=s)

## learnt 95th
mnist_ll_errrfill, = errorfill(depths, te_ll_means_d_MNIST , te_ll_stds_d_MNIST, color=c[2], alpha_fill=0.3, ax=ax[1, 0], lw=lw,
                           linestyle='-')
ax[1, 0].scatter(depths, te_ll_means_d_MNIST, c=c[2], s=s)

fmnist_ll_errrfill, = errorfill(depths, te_ll_means_d_FashionMNIST , te_ll_stds_d_FashionMNIST, color=c[2], alpha_fill=0.3, ax=ax[1, 1], lw=lw,
                           linestyle='-')
ax[1, 1].scatter(depths, te_ll_means_d_FashionMNIST, c=c[2], s=s)

svhn_ll_errrfill, = errorfill(depths, te_ll_means_d_SVHN , te_ll_stds_d_SVHN, color=c[2], alpha_fill=0.3, ax=ax[1, 2], lw=lw,
                           linestyle='-')
ax[1, 2].scatter(depths, te_ll_means_d_SVHN, c=c[2], s=s)

a_lgd = [svhn_det_ll_errrfill, svhn_ll_errrfill]
lgd = ax[1, 2].legend(a_lgd,
['DDN', 'LDN-95'],
                prop={'size': fs-1, 'weight': 'normal'}, frameon=True, markerscale=markersize, loc='lower right')

## learnt 95th
mnist_ll_errrfill, = errorfill(depths, te_ll_means_d_MNIST , te_ll_stds_d_MNIST, color=c[2], alpha_fill=0.3, ax=ax[2, 0], lw=lw,
                           linestyle='-')
ax[2, 0].scatter(depths, te_ll_means_d_MNIST, c=c[2], s=s)

fmnist_ll_errrfill, = errorfill(depths, te_ll_means_d_FashionMNIST , te_ll_stds_d_FashionMNIST, color=c[2], alpha_fill=0.3, ax=ax[2, 1], lw=lw,
                           linestyle='-')
ax[2, 1].scatter(depths, te_ll_means_d_FashionMNIST, c=c[2], s=s)

svhn_ll_errrfill, = errorfill(depths, te_ll_means_d_SVHN , te_ll_stds_d_SVHN, color=c[2], alpha_fill=0.3, ax=ax[2, 2], lw=lw,
                           linestyle='-')
ax[2, 2].scatter(depths, te_ll_means_d_SVHN, c=c[2], s=s)

## full D
mnist_full_err_errrfill, = errorfill(depths, te_ll_means_MNIST, te_ll_stds_MNIST, color=c[5], alpha_fill=0.3, ax=ax[2, 0], lw=lw,
                           linestyle='-')
ax[2, 0].scatter(depths, te_ll_means_MNIST, c=c[5], s=s)

fmnist_full_ll_errrfill, = errorfill(depths, te_ll_means_FashionMNIST, te_ll_stds_FashionMNIST, color=c[5], alpha_fill=0.3, ax=ax[2, 1], lw=lw,
                           linestyle='-')
ax[2, 1].scatter(depths, te_ll_means_FashionMNIST, c=c[5], s=s)

svhn_full_ll_errrfill, = errorfill(depths, te_ll_means_SVHN , te_ll_stds_SVHN, color=c[5], alpha_fill=0.3, ax=ax[2, 2], lw=lw,
                           linestyle='-')
ax[2, 2].scatter(depths, te_ll_means_SVHN, c=c[5], s=s)

## learnt exp
mnist_exp_ll_errrfill, = errorfill(depths, te_ll_means_d_exp_MNIST , te_ll_stds_d_exp_MNIST, color=c[4], alpha_fill=0.3, ax=ax[2, 0], lw=lw,
                           linestyle='-')
ax[2, 0].scatter(depths, te_ll_means_d_exp_MNIST, c=c[4], s=s)

fmnist_exp_ll_errrfill, = errorfill(depths, te_ll_means_d_exp_FashionMNIST , te_ll_stds_d_exp_FashionMNIST, color=c[4], alpha_fill=0.3, ax=ax[2, 1], lw=lw,
                           linestyle='-')
ax[2, 1].scatter(depths, te_ll_means_d_exp_FashionMNIST, c=c[4], s=s)

svhn_exp_ll_errrfill, = errorfill(depths, te_ll_means_d_exp_SVHN , te_ll_stds_d_exp_SVHN, color=c[4], alpha_fill=0.3, ax=ax[2, 2], lw=lw,
                           linestyle='-')
ax[2, 2].scatter(depths, te_ll_means_d_exp_SVHN, c=c[4], s=s)

a_lgd = [svhn_exp_ll_errrfill, svhn_ll_errrfill, svhn_full_ll_errrfill]
lgd = ax[2, 2].legend(a_lgd,
['LDN-$\mathbb{E}$', 'LDN-95', 'LDN-full'],
                prop={'size': fs-1, 'weight': 'normal'}, frameon=True, markerscale=markersize, loc='lower right')

### errors
## determinisitc
mnist_det_err_errrfill, = errorfill(depths, perdd_te_err_means_MNIST, perdd_te_err_stds_MNIST, color=c[3], alpha_fill=0.3, ax=ax[3, 0], lw=lw,
                           linestyle='-')
ax[3, 0].scatter(depths, perdd_te_err_means_MNIST, c=c[3], s=s)

fmnist_det_err_errrfill, = errorfill(depths, perdd_te_err_means_FashionMNIST, perdd_te_err_stds_FashionMNIST, color=c[3], alpha_fill=0.3, ax=ax[3, 1], lw=lw,
                           linestyle='-')
ax[3, 1].scatter(depths, perdd_te_err_means_FashionMNIST, c=c[3], s=s)

svhn_det_err_errrfill, = errorfill(depths, perdd_te_err_means_SVHN, perdd_te_err_stds_SVHN, color=c[3], alpha_fill=0.3, ax=ax[3, 2], lw=lw,
                           linestyle='-')
ax[3, 2].scatter(depths, perdd_te_err_means_SVHN, c=c[3], s=s)

## learnt 95th
mnist_err_errrfill, = errorfill(depths, te_err_means_d_MNIST , te_err_stds_d_MNIST, color=c[2], alpha_fill=0.3, ax=ax[3, 0], lw=lw,
                           linestyle='-')
ax[3, 0].scatter(depths, te_err_means_d_MNIST, c=c[2], s=s)

fmnist_err_errrfill, = errorfill(depths, te_err_means_d_FashionMNIST , te_err_stds_d_FashionMNIST, color=c[2], alpha_fill=0.3, ax=ax[3, 1], lw=lw,
                           linestyle='-')
ax[3, 1].scatter(depths, te_err_means_d_FashionMNIST, c=c[2], s=s)

svhn_err_errrfill, = errorfill(depths, te_err_means_d_SVHN , te_err_stds_d_SVHN, color=c[2], alpha_fill=0.3, ax=ax[3, 2], lw=lw,
                           linestyle='-')
ax[3, 2].scatter(depths, te_err_means_d_SVHN, c=c[2], s=s)

a_lgd = [svhn_det_err_errrfill, svhn_err_errrfill]
lgd = ax[3, 2].legend(a_lgd,
['DDN', 'LDN-95'],
                prop={'size': fs-1, 'weight': 'normal'}, frameon=True, markerscale=markersize, loc='upper right')

## learnt 95th
mnist_err_errrfill, = errorfill(depths, te_err_means_d_MNIST , te_err_stds_d_MNIST, color=c[2], alpha_fill=0.3, ax=ax[4, 0], lw=lw,
                           linestyle='-')
ax[4, 0].scatter(depths, te_err_means_d_MNIST, c=c[2], s=s)

fmnist_err_errrfill, = errorfill(depths, te_err_means_d_FashionMNIST , te_err_stds_d_FashionMNIST, color=c[2], alpha_fill=0.3, ax=ax[4, 1], lw=lw,
                           linestyle='-')
ax[4, 1].scatter(depths, te_err_means_d_FashionMNIST, c=c[2], s=s)

svhn_err_errrfill, = errorfill(depths, te_err_means_d_SVHN , te_err_stds_d_SVHN, color=c[2], alpha_fill=0.3, ax=ax[4, 2], lw=lw,
                           linestyle='-')
ax[4, 2].scatter(depths, te_err_means_d_SVHN, c=c[2], s=s)

## full D
mnist_full_err_errrfill, = errorfill(depths, te_err_means_MNIST, te_err_stds_MNIST, color=c[5], alpha_fill=0.3, ax=ax[4, 0], lw=lw,
                           linestyle='-')
ax[4, 0].scatter(depths, te_err_means_MNIST, c=c[5], s=s)

fmnist_full_err_errrfill, = errorfill(depths, te_err_means_FashionMNIST, te_err_stds_FashionMNIST, color=c[5], alpha_fill=0.3, ax=ax[4, 1], lw=lw,
                           linestyle='-')
ax[4, 1].scatter(depths, te_err_means_FashionMNIST, c=c[5], s=s)

svhn_full_err_errrfill, = errorfill(depths, te_err_means_SVHN , te_err_stds_SVHN, color=c[5], alpha_fill=0.3, ax=ax[4, 2], lw=lw,
                           linestyle='-')
ax[4, 2].scatter(depths, te_err_means_SVHN, c=c[5], s=s)

## learnt exp
mnist_exp_err_errrfill, = errorfill(depths, te_err_means_d_exp_MNIST , te_err_stds_d_exp_MNIST, color=c[4], alpha_fill=0.3, ax=ax[4, 0], lw=lw,
                           linestyle='-')
ax[4, 0].scatter(depths, te_err_means_d_exp_MNIST, c=c[4], s=s)

fmnist_exp_err_errrfill, = errorfill(depths, te_err_means_d_exp_FashionMNIST , te_err_stds_d_exp_FashionMNIST, color=c[4], alpha_fill=0.3, ax=ax[4, 1], lw=lw,
                           linestyle='-')
ax[4, 1].scatter(depths, te_err_means_d_exp_FashionMNIST, c=c[4], s=s)

svhn_exp_err_errrfill, = errorfill(depths, te_err_means_d_exp_SVHN , te_err_stds_d_exp_SVHN, color=c[4], alpha_fill=0.3, ax=ax[4, 2], lw=lw,
                           linestyle='-')
ax[4, 2].scatter(depths, te_err_means_d_exp_SVHN, c=c[4], s=s)

a_lgd = [svhn_exp_err_errrfill, svhn_err_errrfill, svhn_full_err_errrfill]
lgd = ax[4, 2].legend(a_lgd,
['LDN-$\mathbb{E}$', 'LDN-95', 'LDN-full'],
                prop={'size': fs-1, 'weight': 'normal'}, frameon=True, markerscale=markersize, loc='upper right')

plt.tight_layout()
plt.savefig(plot_savedir + 'image_depth_dist_exps2.pdf', format='pdf', bbox_inches='tight')


## Calibration plots

In [ ]:
from src.utils import np_get_one_hot

def plot_calibration_curve_probs(probs, y_test, n_bins=10, dpi=200, grid_alph=0.3, yax=True, title=None, ax=None, xax=True):
    bin_limits = np.linspace(0, 1, n_bins + 1)

    bin_step = bin_limits[1] - bin_limits[0]
    bin_centers = bin_limits[:-1] + bin_step / 2

    bin_probs = []
    bin_counts_list = []
    for i in range(probs.shape[0]):
        all_preds = probs[i]
        pred_class = np.argmax(all_preds, axis=1)

        expanded_preds = np.reshape(all_preds, -1)
        # These reshapes on the one hot vectors count every possible class as a different prediction
        pred_class_OH_expand = np.reshape(np_get_one_hot(pred_class, all_preds.shape[1]), -1)
        targets_class_OH_expand = np.reshape(np_get_one_hot(y_test.reshape(-1, 1).astype(int), all_preds.shape[1]), -1)
        correct_vec = (targets_class_OH_expand * (pred_class_OH_expand == targets_class_OH_expand)).astype(int)

        bin_idxs = np.digitize(expanded_preds, bin_limits, right=True) - 1

        bin_counts = np.ones(n_bins)
        bin_corrects = np.zeros(n_bins)
        for nbin in range(n_bins):
            bin_counts[nbin] = np.sum((bin_idxs == nbin).astype(int))
            bin_corrects[nbin] = np.sum(correct_vec[bin_idxs == nbin])

        bin_probs.append(bin_corrects / bin_counts)
        bin_counts_list.append(bin_counts)

    bin_probs = np.array(bin_probs) # (4, 10)
    
    bin_counts = np.array(bin_counts_list) # (4, 10)
    bin_totals = np.sum(bin_counts, axis=1, keepdims=True) # (4, 1)
    bin_errs = bin_probs - np.arange(0.1, 1.1, 0.1) # (4, 10)
    eces = np.sum(bin_counts / bin_totals * np.abs(bin_errs), axis=1) # (4,)
    ece_mean = np.mean(eces * 100)
    ece_std = np.std(eces * 100)


    bin_prob_means = np.mean(bin_probs, axis=0)
    bin_prob_stds = np.std(bin_probs, axis=0)
    

    if ax is None:
        plt.figure(dpi=dpi)
        ax = plt.gca()
    bar_ret = ax.bar(bin_centers, bin_prob_means, 1 / n_bins, edgecolor='k', alpha=0.9, yerr=bin_prob_stds)
    ax.plot(np.linspace(0, 1, 20), np.linspace(0, 1, 20), '--', c='k')

    if xax:
        ax.set_xlabel('predicted probability')
    if yax:
        ax.set_ylabel('correct proportion')

    if title is not None:
        ax.set_title(title + ", $ %6.2f \pm %6.3f $" % (ece_mean, ece_std))
    ax.set_xticks(bin_limits[::2])
    ax.yaxis.grid(alpha=grid_alph)
    ax.xaxis.grid(alpha=grid_alph)
    ax.set_ylim((0, 1))
    if probs.shape[2] == 2:
        ax.set_xlim((0.5, 1))
    else:
        ax.set_xlim((0, 1))

    return bar_ret

In [ ]:
n_layers = 50

### Generate data for SVHN

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 3
output_dim = 10
outer_width = 64
inner_width = 32

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

trainset = datasets.SVHN(root='../data', split="train", download=True, transform=transform_train)
valset = datasets.SVHN(root='../data', split="test", download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
prior_probs = 0.85 ** (1 + np.arange(n_layers + 1))
prior_probs = prior_probs / prior_probs.sum()
prob_model = variational_categorical(n_layers, prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                inner_width, n_layers, prob_model)  
N_train = len(trainset)
lr = 1e-1
net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

our_probs_SVHN = []
full_int_probs_SVHN = []

for i in [0, 1, 2, 3]:
    print(i)
    our_probs_SVHN.append([])
    full_int_probs_SVHN.append([])

    net.load('../saves/logs/CNN_BNN_SVHN_cat/max_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

    prbs = net.model.prob_model.get_q_probs().data.cpu().numpy()
    cuttoff = np.max(prbs)*0.95
    # chosen_depth = np.sum(prbs * np.arange(net.model.n_layers + 1))
    chosen_depth = np.argmax(prbs > cuttoff)


    ys_SVHN = []

    for x, y in valloader:
        ys_SVHN.append(y.data.numpy())

        full_int_probs_SVHN[-1].append(net.sample_predict(x).sum(dim=0).data.cpu().numpy())
        our_probs_SVHN[-1].append(net.partial_predict(x, depth=chosen_depth).sum(dim=0).data.cpu().numpy())

    our_probs_SVHN[-1] = np.concatenate(our_probs_SVHN[-1])
    full_int_probs_SVHN[-1] = np.concatenate(full_int_probs_SVHN[-1])
    ys_SVHN = np.concatenate(ys_SVHN)

# Load Det Model #######

probs = np.zeros(n_layers + 1)
probs[-1] = 1

prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                inner_width, n_layers, prob_model) 
net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None)

det_probs_SVHN = []
for i in [0, 1, 2, 3]:
    net.load('../saves/logs/CNN_BNN_SVHN_deterministic_cat/deterministic_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
         + '/theta_best.dat')

    det_probs_SVHN.append([])

    for x, _ in valloader:
        det_probs_SVHN[-1].append(net.sample_predict(x).sum(dim=0).data.cpu().numpy())

    det_probs_SVHN[-1] = np.concatenate(det_probs_SVHN[-1])


det_probs_SVHN = np.array(det_probs_SVHN)
our_probs_SVHN = np.array(our_probs_SVHN)
full_int_probs_SVHN = np.array(full_int_probs_SVHN)



### Save data for SVHN

In [ ]:
# to_save = (
#     det_probs_SVHN,
#     our_probs_SVHN,
#     full_int_probs_SVHN,
#     ys_SVHN
# )


# with open('../saves/paper_plots/spiral_callibration_SVHN.pkl','wb') as f:
#      pickle.dump(to_save, f)

### Load data for SVHN

In [ ]:
with open('../saves/paper_plots/spiral_callibration_SVHN.pkl','rb') as f:
     (
        det_probs_SVHN,
        our_probs_SVHN,
        full_int_probs_SVHN,
        ys_SVHN
    ) = pickle.load(f)

### Gen plots for SVHN

In [ ]:
dpi=500

fix, ax = plt.subplots(nrows=1, ncols=3, dpi=dpi,
                       figsize=(1*text_width, text_width*0.45), sharey='row')

ax[0].set_aspect('equal')
ax[1].set_aspect('equal')
ax[2].set_aspect('equal')

plot_calibration_curve_probs(det_probs_SVHN, ys_SVHN, n_bins=10, dpi=200, grid_alph=0.3, yax=True, title='DDN', ax=ax[0], xax=False)
plot_calibration_curve_probs(our_probs_SVHN, ys_SVHN, n_bins=10, dpi=200, grid_alph=0.3, yax=False, title='LDN, $d \in [0, d_{opt}]$', ax=ax[1])
plot_calibration_curve_probs(full_int_probs_SVHN, ys_SVHN, n_bins=10, dpi=200, grid_alph=0.3, yax=False, title='LDN, $d \in [0, D]$', ax=ax[2], xax=False)

plt.tight_layout()



plt.savefig(plot_savedir + 'callibration_SVHN.pdf', format='pdf', bbox_inches='tight')

### Generate plot data for MNIST

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.MNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
prior_probs = 0.85 ** (1 + np.arange(n_layers + 1))
prior_probs = prior_probs / prior_probs.sum()
prob_model = variational_categorical(n_layers, prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                inner_width, n_layers, prob_model)  
N_train = len(trainset)
lr = 1e-1
net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

our_probs_MNIST = []
full_int_probs_MNIST = []

for i in [0, 1, 2, 3]:
    our_probs_MNIST.append([])
    full_int_probs_MNIST.append([])

    net.load('../saves/logs/CNN_BNN_MNIST_cat/max_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

    prbs = net.model.prob_model.get_q_probs().data.cpu().numpy()
    cuttoff = np.max(prbs)*0.95
    # chosen_depth = np.sum(prbs * np.arange(net.model.n_layers + 1))
    chosen_depth = np.argmax(prbs > cuttoff)


    ys_MNIST = []

    for x, y in valloader:
        ys_MNIST.append(y.data.numpy())

        full_int_probs_MNIST[-1].append(net.sample_predict(x).sum(dim=0).data.cpu().numpy())
        our_probs_MNIST[-1].append(net.partial_predict(x, depth=chosen_depth).sum(dim=0).data.cpu().numpy())

    our_probs_MNIST[-1] = np.concatenate(our_probs_MNIST[-1])
    full_int_probs_MNIST[-1] = np.concatenate(full_int_probs_MNIST[-1])
    ys_MNIST = np.concatenate(ys_MNIST)

# Load Det Model #######

probs = np.zeros(n_layers + 1)
probs[-1] = 1

prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                inner_width, n_layers, prob_model) 
net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None)

det_probs_MNIST = []
for i in [0, 1, 2, 3]:
    net.load('../saves/logs/CNN_BNN_MNIST_deterministic_cat/deterministic_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
         + '/theta_best.dat')

    det_probs_MNIST.append([])

    for x, _ in valloader:
        det_probs_MNIST[-1].append(net.sample_predict(x).sum(dim=0).data.cpu().numpy())

    det_probs_MNIST[-1] = np.concatenate(det_probs_MNIST[-1])


det_probs_MNIST = np.array(det_probs_MNIST)
our_probs_MNIST = np.array(our_probs_MNIST)
full_int_probs_MNIST = np.array(full_int_probs_MNIST)

### Save plot data for MNIST

In [ ]:
# to_save = (
#     det_probs_MNIST,
#     our_probs_MNIST,
#     full_int_probs_MNIST,
#     ys_MNIST
# )


# with open('../saves/paper_plots/spiral_callibration_MNIST.pkl','wb') as f:
#      pickle.dump(to_save, f)

### Load plot data for MNIST

In [ ]:
with open('../saves/paper_plots/spiral_callibration_MNIST.pkl','rb') as f:
     (
        det_probs_MNIST,
        our_probs_MNIST,
        full_int_probs_MNIST,
        ys_MNIST
    ) = pickle.load(f)

### Gen plots for MNIST

In [ ]:
dpi=800

fix, ax = plt.subplots(nrows=1, ncols=3, dpi=dpi,
                       figsize=(1*text_width, text_width*0.45), sharey='row')

ax[0].set_aspect('equal')
ax[1].set_aspect('equal')
ax[2].set_aspect('equal')

plot_calibration_curve_probs(det_probs_MNIST, ys_MNIST, n_bins=10, dpi=200, grid_alph=0.3, yax=True, title='DDN', ax=ax[0], xax=False)
plot_calibration_curve_probs(our_probs_MNIST, ys_MNIST, n_bins=10, dpi=200, grid_alph=0.3, yax=False, title='LDN, $d \in [0, d_{opt}]$', ax=ax[1])
plot_calibration_curve_probs(full_int_probs_MNIST, ys_MNIST, n_bins=10, dpi=200, grid_alph=0.3, yax=False, title='LDN, $d \in [0, D]$', ax=ax[2], xax=False)

plt.tight_layout()



plt.savefig(plot_savedir + 'callibration_MNIST.pdf', format='pdf', bbox_inches='tight')

### Generate plot data for FashionMNIST

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

trainset = datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
# LDN

prior_probs = 0.85 ** (1 + np.arange(n_layers + 1))
prior_probs = prior_probs / prior_probs.sum()
prob_model = variational_categorical(n_layers, prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                inner_width, n_layers, prob_model)  
N_train = len(trainset)
lr = 1e-1
net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

our_probs_FashionMNIST = []
full_int_probs_FashionMNIST = []

for i in [0, 1, 2, 3]:
    print(i)
    our_probs_FashionMNIST.append([])
    full_int_probs_FashionMNIST.append([])

    net.load('../saves/logs/CNN_BNN_FashionMNIST_cat/max_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

    prbs = net.model.prob_model.get_q_probs().data.cpu().numpy()
    cuttoff = np.max(prbs)*0.95
    # chosen_depth = np.sum(prbs * np.arange(net.model.n_layers + 1))
    chosen_depth = np.argmax(prbs > cuttoff)


    ys_FashionMNIST = []

    for x, y in valloader:
        ys_FashionMNIST.append(y.data.numpy())

        full_int_probs_FashionMNIST[-1].append(net.sample_predict(x).sum(dim=0).data.cpu().numpy())
        our_probs_FashionMNIST[-1].append(net.partial_predict(x, depth=chosen_depth).sum(dim=0).data.cpu().numpy())

    our_probs_FashionMNIST[-1] = np.concatenate(our_probs_FashionMNIST[-1])
    full_int_probs_FashionMNIST[-1] = np.concatenate(full_int_probs_FashionMNIST[-1])
    ys_FashionMNIST = np.concatenate(ys_FashionMNIST)

# DDN

probs = np.zeros(n_layers + 1)
probs[-1] = 1

prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                inner_width, n_layers, prob_model) 
net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None)

det_probs_FashionMNIST = []
for i in [0, 1, 2, 3]:
    net.load('../saves/logs/CNN_BNN_FashionMNIST_deterministic_cat/deterministic_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
         + '/theta_best.dat')

    det_probs_FashionMNIST.append([])

    for x, _ in valloader:
        det_probs_FashionMNIST[-1].append(net.sample_predict(x).sum(dim=0).data.cpu().numpy())

    det_probs_FashionMNIST[-1] = np.concatenate(det_probs_FashionMNIST[-1])


det_probs_FashionMNIST = np.array(det_probs_FashionMNIST)
our_probs_FashionMNIST = np.array(our_probs_FashionMNIST)
full_int_probs_FashionMNIST = np.array(full_int_probs_FashionMNIST)

### Save plot data for FashionMNIST

In [ ]:
# to_save = (
#     det_probs_FashionMNIST,
#     our_probs_FashionMNIST,
#     full_int_probs_FashionMNIST,
#     ys_FashionMNIST
# )


# with open('../saves/paper_plots/spiral_callibration_FashionMNIST.pkl','wb') as f:
#      pickle.dump(to_save, f)

### Load plot data for FashionMNIST


In [ ]:
with open('../saves/paper_plots/spiral_callibration_FashionMNIST.pkl','rb') as f:
     (
        det_probs_FashionMNIST,
        our_probs_FashionMNIST,
        full_int_probs_FashionMNIST,
        ys_FashionMNIST
    ) = pickle.load(f)

### Generate plots for FashionMNIST

In [ ]:
dpi=800

fix, ax = plt.subplots(nrows=1, ncols=3, dpi=dpi,
                       figsize=(1*text_width, text_width*0.45), sharey='row')

ax[0].set_aspect('equal')
ax[1].set_aspect('equal')
ax[2].set_aspect('equal')

plot_calibration_curve_probs(det_probs_FashionMNIST, ys_FashionMNIST, n_bins=10, dpi=200, grid_alph=0.3, yax=True, title='DDN', ax=ax[0], xax=False)
plot_calibration_curve_probs(our_probs_FashionMNIST, ys_FashionMNIST, n_bins=10, dpi=200, grid_alph=0.3, yax=False, title='LDN, $d \in [0, d_{opt}]$', ax=ax[1])
plot_calibration_curve_probs(full_int_probs_FashionMNIST, ys_FashionMNIST, n_bins=10, dpi=200, grid_alph=0.3, yax=False, title='LDN, $d \in [0, D]$', ax=ax[2], xax=False)

plt.tight_layout()



plt.savefig(plot_savedir + 'callibration_FashionMNIST.pdf', format='pdf', bbox_inches='tight')

## Timing experiments

In [ ]:
import time

### Generate plot data for SVHN

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 3
output_dim = 10
outer_width = 64
inner_width = 32

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.431, 0.430, 0.446), (0.197, 0.198, 0.199))
])

trainset = datasets.SVHN(root='../data', split="train", download=True, transform=transform_train)
valset = datasets.SVHN(root='../data', split="test", download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
valset_partial_pred_times_SVHN = []
valset_sample_pred_times_SVHN = []
valset_det_pred_times_SVHN = []

for n_layers in [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]:
    prior_probs = 0.85 ** (1 + np.arange(n_layers + 1))
    prior_probs = prior_probs / prior_probs.sum()
    prob_model = variational_categorical(n_layers, prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
    model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                        inner_width, n_layers, prob_model)  
    N_train = len(trainset)
    lr = 1e-1
    net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

    for i in range(4):
        valset_partial_pred_times_SVHN.append([])
        valset_sample_pred_times_SVHN.append([])
        valset_det_pred_times_SVHN.append([])
        
        net.load('../saves/logs/CNN_BNN_SVHN_cat/max_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

        probs = net.model.prob_model.get_q_probs().data.cpu().numpy()
        cuttoff = np.max(probs)*0.95
        # depth_exp = np.sum(probs * np.arange(net.model.n_layers + 1))
        chosen_depth = np.argmax(probs > cuttoff)

        for _ in range(10):
            tic = time.time()
            for x, _ in valloader:
                if cuda:
                    y = y.cuda()

                prbs = net.partial_predict(x, depth=chosen_depth).sum(dim=0)
                pred = prbs.max(dim=1, keepdim=False)[1]

            toc = time.time()
            valset_partial_pred_times_SVHN[-1].append(toc - tic)

        for _ in range(10):
            tic = time.time()
            for x, _ in valloader:
                if cuda:
                    y = y.cuda()

                prbs = net.sample_predict(x).sum(dim=0)
                pred = prbs.max(dim=1, keepdim=False)[1]

            toc = time.time()
            valset_sample_pred_times_SVHN[-1].append(toc - tic)

        probs = np.zeros(n_layers + 1)
        probs[-1] = 1

        prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                        inner_width, n_layers, prob_model) 
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None)

        
        net.load('../saves/logs/CNN_BNN_SVHN_deterministic_cat/deterministic_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

        for _ in range(10):
            tic = time.time()
            for x, _ in valloader:
                if cuda:
                    y = y.cuda()

                prbs = net.sample_predict(x).sum(dim=0)
                pred = prbs.max(dim=1, keepdim=False)[1]

            toc = time.time()
            valset_det_pred_times_SVHN[-1].append(toc - tic)

    valset_partial_pred_times_SVHN[-1] = np.mean(np.array(valset_partial_pred_times_SVHN[-1]), axis=1)
    valset_sample_pred_times_SVHN[-1] = np.mean(np.array(valset_sample_pred_times_SVHN[-1]), axis=1)
    valset_det_pred_times_SVHN[-1] = np.mean(np.array(valset_det_pred_times_SVHN[-1]), axis=1)



In [ ]:
### Save plot data

# to_save = (
#     valset_partial_pred_times_SVHN,
#     valset_sample_pred_times_SVHN,
#     valset_det_pred_times_SVHN
# )


# with open('../saves/paper_plots/times_SVHN.pkl','wb') as f:
#      pickle.dump(to_save, f)

### Generate plot data for MNIST

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.MNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
valset_partial_pred_times_MNIST = []
valset_sample_pred_times_MNIST = []
valset_det_pred_times_MNIST = []

for n_layers in [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]:
    prior_probs = 0.85 ** (1 + np.arange(n_layers + 1))
    prior_probs = prior_probs / prior_probs.sum()
    prob_model = variational_categorical(n_layers, prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
    model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                        inner_width, n_layers, prob_model)  
    N_train = len(trainset)
    lr = 1e-1
    net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

    valset_partial_pred_times_MNIST.append([])
    valset_sample_pred_times_MNIST.append([])
    valset_det_pred_times_MNIST.append([])

    for i in range(4):
        net.load('../saves/logs/CNN_BNN_MNIST_cat/max_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

        probs = net.model.prob_model.get_q_probs().data.cpu().numpy()
        cuttoff = np.max(probs)*0.95
        # depth_exp = np.sum(probs * np.arange(net.model.n_layers + 1))
        chosen_depth = np.argmax(probs > cuttoff)

        tic = time.time()
        for x, _ in valloader:
            if cuda:
                y = y.cuda()

            prbs = net.partial_predict(x, depth=chosen_depth).sum(dim=0)
            pred = prbs.max(dim=1, keepdim=False)[1]

        toc = time.time()
        valset_partial_pred_times_MNIST[-1].append(toc - tic)

        tic = time.time()
        for x, _ in valloader:
            if cuda:
                y = y.cuda()

            prbs = net.sample_predict(x).sum(dim=0)
            pred = prbs.max(dim=1, keepdim=False)[1]

        toc = time.time()
        valset_sample_pred_times_MNIST[-1].append(toc - tic)

        probs = np.zeros(n_layers + 1)
        probs[-1] = 1

        prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                        inner_width, n_layers, prob_model) 
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None)

        
        net.load('../saves/logs/CNN_BNN_MNIST_deterministic_cat/deterministic_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

        tic = time.time()
        for x, _ in valloader:
            if cuda:
                y = y.cuda()

            prbs = net.sample_predict(x).sum(dim=0)
            pred = prbs.max(dim=1, keepdim=False)[1]

        toc = time.time()
        valset_det_pred_times_MNIST[-1].append(toc - tic)

    valset_partial_pred_times_MNIST[-1] = np.array(valset_partial_pred_times_MNIST[-1])
    valset_sample_pred_times_MNIST[-1] = np.array(valset_sample_pred_times_MNIST[-1])
    valset_det_pred_times_MNIST[-1] = np.array(valset_det_pred_times_MNIST[-1])



In [ ]:
# to_save = (
#     valset_partial_pred_times_MNIST,
#     valset_sample_pred_times_MNIST,
#     valset_det_pred_times_MNIST
# )


# with open('../saves/paper_plots/times_MNIST.pkl','wb') as f:
#      pickle.dump(to_save, f)

### Generate plot data for FashionMNIST

In [ ]:
cuda = torch.cuda.is_available()

input_chan = 1
output_dim = 10
outer_width = 64
inner_width = 32

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])

trainset = datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform_train)
valset = datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform_test)

batch_size = 128
if cuda:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                            num_workers=3)

else:
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                num_workers=3)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                            num_workers=3)

In [ ]:
valset_partial_pred_times_FashionMNIST = []
valset_sample_pred_times_FashionMNIST = []
valset_det_pred_times_FashionMNIST = []

for n_layers in [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]:
    prior_probs = 0.85 ** (1 + np.arange(n_layers + 1))
    prior_probs = prior_probs / prior_probs.sum()
    prob_model = variational_categorical(n_layers, prior_probs, temp=0.1, eps=1e-10, cuda=cuda)
    model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                        inner_width, n_layers, prob_model)  
    N_train = len(trainset)
    lr = 1e-1
    net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None) 

    valset_partial_pred_times_FashionMNIST.append([])
    valset_sample_pred_times_FashionMNIST.append([])
    valset_det_pred_times_FashionMNIST.append([])

    for i in range(4):
        net.load('../saves/logs/CNN_BNN_FashionMNIST_cat/max_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

        probs = net.model.prob_model.get_q_probs().data.cpu().numpy()
        cuttoff = np.max(probs)*0.95
        # depth_exp = np.sum(probs * np.arange(net.model.n_layers + 1))
        chosen_depth = np.argmax(probs > cuttoff)

        tic = time.time()
        for x, _ in valloader:
            if cuda:
                y = y.cuda()

            prbs = net.partial_predict(x, depth=chosen_depth).sum(dim=0)
            pred = prbs.max(dim=1, keepdim=False)[1]

        toc = time.time()
        valset_partial_pred_times_FashionMNIST[-1].append(toc - tic)

        tic = time.time()
        for x, _ in valloader:
            if cuda:
                y = y.cuda()

            prbs = net.sample_predict(x).sum(dim=0)
            pred = prbs.max(dim=1, keepdim=False)[1]

        toc = time.time()
        valset_sample_pred_times_FashionMNIST[-1].append(toc - tic)

        probs = np.zeros(n_layers + 1)
        probs[-1] = 1

        prob_model = fixed_probs(n_layers, probs=probs, distribution_name='cat', cuda=True)
        model = arq_uncert_conv2d_resnet(input_chan, output_dim, outer_width, 
                                        inner_width, n_layers, prob_model) 
        net = MF_BNN_cat(model, N_train, lr=lr, cuda=cuda, schedule=None)

        
        net.load('../saves/logs/CNN_BNN_FashionMNIST_deterministic_cat/deterministic_depth_scan/' + str(n_layers) + '/version_%d/models' % i 
            + '/theta_best.dat')

        tic = time.time()
        for x, _ in valloader:
            if cuda:
                y = y.cuda()

            prbs = net.sample_predict(x).sum(dim=0)
            pred = prbs.max(dim=1, keepdim=False)[1]

        toc = time.time()
        valset_det_pred_times_FashionMNIST[-1].append(toc - tic)

    valset_partial_pred_times_FashionMNIST[-1] = np.array(valset_partial_pred_times_FashionMNIST[-1])
    valset_sample_pred_times_FashionMNIST[-1] = np.array(valset_sample_pred_times_FashionMNIST[-1])
    valset_det_pred_times_FashionMNIST[-1] = np.array(valset_det_pred_times_FashionMNIST[-1])



In [ ]:
# to_save = (
#     valset_partial_pred_times_FashionMNIST,
#     valset_sample_pred_times_FashionMNIST,
#     valset_det_pred_times_FashionMNIST
# )


# with open('../saves/paper_plots/times_FashionMNIST.pkl','wb') as f:
#      pickle.dump(to_save, f)

### Load plot data

In [ ]:
with open('../saves/paper_plots/times_SVHN.pkl','rb') as f:
     (
        valset_partial_pred_times_SVHN,
        valset_sample_pred_times_SVHN,
        valset_det_pred_times_SVHN
     ) = pickle.load(f)

with open('../saves/paper_plots/times_MNIST.pkl','rb') as f:
     (
        valset_partial_pred_times_MNIST,
        valset_sample_pred_times_MNIST,
        valset_det_pred_times_MNIST
     ) = pickle.load(f)

with open('../saves/paper_plots/times_FashionMNIST.pkl','rb') as f:
     (
        valset_partial_pred_times_FashionMNIST,
        valset_sample_pred_times_FashionMNIST,
        valset_det_pred_times_FashionMNIST
     ) = pickle.load(f)

### Generate plots

In [ ]:
speedups_SVHN = 1 - np.array(valset_partial_pred_times_SVHN) / np.array(valset_det_pred_times_SVHN)
speedups_MNIST = 1 - np.array(valset_partial_pred_times_MNIST) / np.array(valset_det_pred_times_MNIST)
speedups_FashionMNIST = 1 - np.array(valset_partial_pred_times_FashionMNIST) / np.array(valset_det_pred_times_FashionMNIST)

In [ ]:
mean_speedup_SVHN, std_speedup_SVHN = get_run_mean_std(speedups_SVHN * 100)
mean_speedup_MNIST, std_speedup_MNIST = get_run_mean_std(speedups_MNIST * 100)
mean_speedup_FashionMNIST, std_speedup_FashionMNIST = get_run_mean_std(speedups_FashionMNIST * 100)

In [ ]:
dpi=800

fig, ax = plt.subplots(nrows=1, ncols=3, dpi=dpi, figsize=(1.1*text_width, 1.8))

depths = [str(x) for x in [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]]

ax[0].bar(depths, mean_speedup_MNIST, edgecolor='k', alpha=0.9, yerr=std_speedup_MNIST)
ax[1].bar(depths, mean_speedup_FashionMNIST, edgecolor='k', alpha=0.9, yerr=std_speedup_FashionMNIST)
ax[2].bar(depths, mean_speedup_SVHN, edgecolor='k', alpha=0.9, yerr=std_speedup_SVHN)

ax[0].set_xticklabels(depths, rotation=45)
ax[1].set_xticklabels(depths, rotation=45)
ax[2].set_xticklabels(depths, rotation=45)

ax[0].set_ylim([np.min(mean_speedup_MNIST - std_speedup_MNIST), np.max(mean_speedup_MNIST + std_speedup_MNIST)])
ax[1].set_ylim([np.min(mean_speedup_FashionMNIST - std_speedup_FashionMNIST), np.max(mean_speedup_FashionMNIST + std_speedup_FashionMNIST)])
ax[2].set_ylim([np.min(mean_speedup_SVHN - std_speedup_SVHN), np.max(mean_speedup_SVHN + std_speedup_SVHN)])

ax[0].set_xlim([-0.5, 15.5])
ax[1].set_xlim([-0.5, 15.5])
ax[2].set_xlim([-0.5, 15.5])

ax[0].yaxis.grid(alpha=0.3)
ax[0].xaxis.grid(alpha=0.3)
ax[1].yaxis.grid(alpha=0.3)
ax[1].xaxis.grid(alpha=0.3)
ax[2].yaxis.grid(alpha=0.3)
ax[2].xaxis.grid(alpha=0.3)

ax[0].set_ylabel("Speedup (\%)")
ax[1].set_xlabel("D (Max depth)")

plt.savefig(plot_savedir + 'image_speedups.pdf', format='pdf', bbox_inches='tight')